In [1]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
from lower_case_dict import d_dict as D_dict
from lower_case_dict import d_describe as D_describe
from traduce import leyenda as tr_leyenda
pd.options.display.max_columns=None
pd.options.display.max_colwidth=300

## Funciones de inspección

In [2]:
def leyenda(col):
    dicc=D_dict
    return tr_leyenda(dicc,col)

In [3]:
def tr(col):
    global df
    return df[col].astype('O').replace(D_dict[col])

## Recuperación y fusión de datasets

In [4]:
# Encuesta de graduados 2019
ruta_g19='data/Graduados_19_modelos.csv'
df_g19=pd.read_csv(ruta_g19,index_col=0)

/home/bea/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (214,275) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_g19.shape

(31651, 275)

In [6]:
# Encuesta de máster 2019
ruta_m19='data/Master_19_modelos.csv'
df_m19=pd.read_csv(ruta_m19,index_col=0)

In [7]:
df_m19.shape

(11483, 274)

In [8]:
df=df_g19
df=df.append(df_m19)
df.shape

(43134, 285)

In [9]:
#df

In [10]:
# Comprobación: 
df.shape[0]==df_g19.shape[0]+df_m19.shape[0] 

True

In [11]:
# Inspección de columnas 
for col in df.columns:
    try: print (col, '\t', D_describe[col])
    except: print ('no', col)

año 	 Año en que se realizó la encuesta
est_encuesta 	 Estudios oficiales objeto de la encuesta (los que definen la población objeto de muestreo)
sexo 	 Mujer u hombre
edad 	 Grupo de edad en el momento de la encuesta
nacionalidad 	 Nacionalidad española u otra
eu27 	 Nacido o no en EU27
titulo_g1 	 Titulación del primer grado universitario, diplomatura, licenciatura o equivalente
rama_g1 	 Rama de conocimiento del primer titulo de grado
n_art_hum 	 Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de artes y humanidades
n_ciencias 	 Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de Ciencias
n_cc_soc_ju 	 Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ciencias sociales y jurídicas
n_ing_arq 	 Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ingeniería y arquitectura
n_cc_salud 	 Número de títulos universitarios (grado, máster, doctorad

In [12]:
for col in df_g19.columns:
    if col not in df_m19.columns:
        print (col)

titulo_g1
varios_grados
master
ambito_g4
rama_g4
titulo_g1_
varios_grados_
master_
ambito_g4_
rama_g4_
n_master_


In [13]:
# Imputación de no informados en encuesta máster 2019:
# 'varios_grados'=1 (Sí) para titulados máster con n_grados>1:
df['varios_grados'].mask((df['est_encuesta']==2) & (df['n_grados']>1),1,inplace=True)
# 'titulo _g1'=1 ('No conocido') para titulados máster con grado:
df['titulo_g1'].mask((df['est_encuesta']==2) & (df['n_grados']>=1),1,inplace=True) 
# 'titulo _g1'=0 ('No tiene') para titulados máster sin grado:
df['titulo_g1'].mask((df['est_encuesta']==2) & (df['n_grados']==0),0,inplace=True)
# 'master'=1 (Sí) para titulados máster
df['master'].mask(df['est_encuesta']==2,1,inplace=True) 

In [14]:
for col in df_m19.columns:
    if col not in df_g19.columns:
        print (col)

titulo_m1
ambito_m1
grado
varios_master
rama_m4
titulo_m1_
ambito_m1_
grado_
varios_master_
rama_m4_


In [15]:
# Imputación de no informados en encuesta grado 2019:
df['varios_master'].mask((df['est_encuesta']==1) & (df['n_master']>1),1,inplace=True)
# 'titulo _m1' 'No conocido' para graduados con máster:
df['titulo_m1'].mask((df['est_encuesta']==1) & (df['n_master']>0),1,inplace=True) 
# 'titulo _m1'=0 ('No tiene') para graduados sin máster:
df['titulo_m1'].mask((df['est_encuesta']==1) & (df['n_master']==0),0,inplace=True)
# 'ambito _m1'=1 ('No conocido') para graduados con máster:
df['ambito_m1'].mask((df['est_encuesta']==1) & (df['n_master']>0),1,inplace=True) 
# 'ambito _m1'=0 ('No tiene') para graduados sin máster:
df['ambito_m1'].mask((df['est_encuesta']==1) & (df['n_master']==0),0,inplace=True)
# 'ambito _m1'=1 ('No conocido') para graduados con máster:
df['ambito_m1'].mask((df['est_encuesta']==1) & (df['n_master']>=1),0,inplace=True)
# 'grado'=1 (Sí) para graduados
df['grado'].mask(df['est_encuesta']==1,1,inplace=True) 

In [16]:
df.shape

(43134, 285)

## Ajuste 'extr_proxy'
'extr' (binaria) es una variable orignal de los dataset. Está referida a los estudios principales (los que hacen a los encuestados elegibles para la muestra).  
  
'extr_proxy' (binaria) es una variable construida a partir de las respuestas a las preguntas sobre dónde cursó otros estudios. 
  
Se modifica la variable 'extr_proxy' para incluir a quienes cursaron al menos parte de sus estudios principales en el extrangero (si no estaban ya registrados por haber cursado otros estudios fuera de España). 

In [17]:
# Referida a los estudios que hacen elegible para la encuesta. 
D_describe['extr'], df['extr'].unique()

('Realizó parte de sus estudios en el extranjero', array([ 0.,  1., nan]))

In [18]:
# Referida al resto de estudios sobre los que se informa. 
# Creada a partir de respuestas sobre dónde cursó otros estudios.
D_describe['extr_proxy'], df['extr_proxy'].unique()

('Cursó al menos parte de sus estudios en el extranjero', array([0, 1]))

In [19]:
df['extr'].sum(), df['extr_proxy'].sum()

(5843.0, 4336)

In [20]:
df[['extr_proxy','extr']].groupby('extr').agg(['value_counts'])

extr_proxy
                value_counts
extr extr_proxy             
0.0  0                 34216
     1                  3052
1.0  0                  4559
     1                  1284

In [21]:
# Modifica 'extr_proxy' para incluir las observaciones para las que 'extr' es 1. 
df['extr_proxy'].mask(df['extr']==1,1,inplace=True)
df['extr_proxy'].sum()

8895

In [22]:
df[['extr_proxy','extr']].groupby('extr').agg(['value_counts'])

extr_proxy
                value_counts
extr extr_proxy             
0.0  0                 34216
     1                  3052
1.0  1                  5843

## Nueva variable 'titulo_ppal'
'titulo_ppal' recoge en una columna única los títulos de los estudios principales de la encuesta, que corresponden distintos niveles de formación universitaria (grado y máster).  
  
Se construye a partir de la información recogida por 'titulo_g1' y 'titulo_m1': las encuestas de grado tienen 'titulo_g1' bien informado y 'titulo_m1' como binaria ('no tiene' o 'no conocido'), y viceversa para las encuestas de titulados en máster. 
  
Se conservan los códigos numéricos de los títulos en ambos casos. Esto es posible gracias a que, para las observaciones bien informadas, los más altos de máster son del orden de los miles y los más bajos de grado son del orden de los diez miles. 

In [23]:
# Códigos de títulos de grado: 
df['titulo_g1'].dropna().astype('int').sort_values().unique()

array([     0,      1,  11101,  11201,  11301,  11401,  11901,  21101,
        21201,  21301,  21302,  21401,  21502,  22201,  22202,  22301,
        22901,  23101,  23102,  23103,  23104,  23201,  23202,  23901,
        31101,  31201,  31202,  31301,  31401,  31402,  31404,  31406,
        32101,  32102,  32201,  41201,  41202,  41301,  41302,  41303,
        41401,  41402,  41403,  41601,  42101,  51101,  51201,  51202,
        51901,  52101,  53101,  53201,  53202,  53203,  53301,  54101,
        54201,  61301,  61901,  71101,  71301,  71302,  71401,  71402,
        71403,  71404,  71405,  71501,  71502,  71503,  71601,  71603,
        71901,  72101,  72102,  72201,  72401,  73101,  73102,  73201,
        73202,  81102,  81103,  81201,  82101,  84101,  91101,  91201,
        91301,  91401,  91402,  91501,  91502,  91503,  91504,  91505,
        91601,  92301, 101401, 101501, 104101, 104103, 109999])

In [24]:
# Códigos de títulos de máster: 
df['titulo_m1'].dropna().astype('int').sort_values().unique()

array([   0,    1,  111,  113,  114,  119,  211,  213,  215,  219,  221,
        222,  223,  229,  231,  232,  239,  311,  312,  313,  314,  319,
        321,  322,  411,  412,  413,  414,  416,  419,  421,  511,  512,
        519,  521,  522,  531,  532,  533,  541,  542,  599,  612,  613,
        619,  711,  712,  713,  714,  715,  716,  719,  721,  722,  731,
        732,  811,  821,  831,  841,  911,  912,  913,  914,  915,  916,
        919,  923, 1013, 1014, 1015, 1022, 1032, 1041])

In [25]:
# Para encuesta de grado ('est_encuesta'=1): 'titulo_m1' binaria
df['titulo_m1'][df['est_encuesta']==1].unique()

array([0., 1.])

In [26]:
# Para encuesta de máster ('est_encuesta'=2): 'titulo_g1' binaria
df['titulo_g1'][df['est_encuesta']==2].unique()

array([1., 0.])

In [27]:
# Se prescinde de los valores 0 y 1 en 'titulo_g1' y 'titulo_m1' (todos los encuestados tienen título bien ...
# ... informado para uno de los dos niveles de formación, y solo para uno).
D_dict['titulo_g1'][0], D_dict['titulo_m1'][0], D_dict['titulo_g1'][1], D_dict['titulo_m1'][1]

('No tiene', 'No tiene', 'Título no conocido', 'Título no conocido')

In [28]:
# Para las encuestas de grado: 
df['titulo_m1'][df['titulo_m1'].isin([0,1])].count()==df[df['est_encuesta']==1].shape[0] # Comprobación

True

In [29]:
# Para las encuestas de máster: 
df['titulo_g1'][df['titulo_g1'].isin([0,1])].count()==df[df['est_encuesta']==2].shape[0] # Comprobación

True

In [30]:
df['titulo_ppal']=df['titulo_g1']
df['titulo_ppal'].mask(df['est_encuesta']==2,df['titulo_m1'],inplace=True)
df['titulo_ppal']=df['titulo_ppal'].astype('int')
df['titulo_ppal'].unique()

array([ 11101,  11201,  11301,  11401,  11901,  21101,  21201,  21301,
        21302,  21401,  21502,  22201,  22202,  22301,  22901,  23101,
        23102,  23103,  23104,  23201,  23202,  23901,  31101,  31201,
        31202,  31301,  31401,  31402,  31404,  31406,  32101,  32102,
        32201,  41201,  41202,  41301,  41302,  41303,  41401,  41402,
        41403,  41601,  42101,  51101,  51201,  51202,  51901,  52101,
        53101,  53201,  53202,  53203,  53301,  54101,  54201,  61301,
        61901,  71101,  71301,  71302,  71401,  71402,  71403,  71404,
        71405,  71501,  71502,  71503,  71601,  71603,  71901,  72101,
        72102,  72201,  72401,  73101,  73102,  73201,  73202,  81102,
        81103,  81201,  82101,  84101,  91101,  91201,  91301,  91401,
        91402,  91501,  91502,  91503,  91504,  91505,  91601,  92301,
       101401, 101501, 104101, 104103, 109999,    916,    114,    612,
          531,    519,    915,    713,    421,   1022,    619,    716,
      

In [31]:
# Comprobación:
df['titulo_ppal'].nunique()==df[~df['titulo_g1'].isin([0,1])]['titulo_g1'].nunique()\
                            +df[~df['titulo_m1'].isin([0,1])]['titulo_m1'].nunique()

True

In [32]:
D_describe['titulo_ppal']='Denominación del título universitario principal '+\
                          '(el que hace a los titulados elegibles para la encuesta en la que participan)'
D_describe['titulo_ppal']

'Denominación del título universitario principal (el que hace a los titulados elegibles para la encuesta en la que participan)'

In [33]:
 def reune_dicc(d1,d2):
    nuevo=d1
    for key in d2: 
        if key not in d1: 
            d1[key]=d2[key]
    return nuevo

In [34]:
D_dict['titulo_ppal']=reune_dicc(D_dict['titulo_g1'],D_dict['titulo_m1'])
df['titulo_ppal_']=df['titulo_ppal'].astype('O').replace(D_dict['titulo_ppal'])
#df['titulo_ppal_'].unique()

## Ajustes en variables de idiomas (nuevas variables)

In [35]:
# Inspección: columnas referidas a idiomas.
col_idiomas=list(filter(lambda col: re.findall('.*idioma.*',col),df.columns))
col_idiomas

['idioma_m1',
 'idioma_m2',
 'n_idiomas',
 'idioma1',
 'idioma1_nivel',
 'idioma1_acred',
 'idioma2',
 'idioma2_nivel',
 'idioma2_acred',
 'idioma3',
 'idioma3_nivel',
 'idioma3_acred',
 'idioma4',
 'idioma4_nivel',
 'idioma4_acred',
 'dbtr_idiomas',
 'vbtr_idiomas',
 'idioma_m1_',
 'idioma_m2_',
 'idioma1_',
 'idioma1_nivel_',
 'idioma1_acred_',
 'idioma2_',
 'idioma2_nivel_',
 'idioma2_acred_',
 'idioma3_',
 'idioma3_nivel_',
 'idioma3_acred_',
 'idioma4_',
 'idioma4_nivel_',
 'idioma4_acred_',
 'dbtr_idiomas_',
 'vbtr_idiomas_']

In [36]:
# Comprobación: solo la categoría 'Otros_idiomas' aparece repetido en columnas de idioma distintas.
check_list=['idioma_m1_','idioma_m2_','idioma1_','idioma2_','idioma3_','idioma4_']
for idioma_a in check_list:
    for idioma_b in [col for col in check_list if col!=idioma_a]:
        print (df[str(idioma_a)][df[str(idioma_a)]==df[str(idioma_b)]].unique())

['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']
['Otros idiomas']


In [37]:
# Inspección: primera lengua materna: 
df['idioma_m1_'].value_counts()

Español          36719
Catalán           3254
Gallego            970
Euskera            938
Valenciano         759
Otros idiomas      143
Inglés              78
Francés             49
Rumano              45
Portugués           35
Árabe               33
Alemán              32
Italiano            28
Ruso                26
Chino               25
Name: idioma_m1_, dtype: int64

In [38]:
df['idioma_m1_'].nunique()

15

In [39]:
# Inspección: segunda lengua materna: 
df['idioma_m2_'].unique()

array([nan, 'Catalán', 'Gallego', 'Español', 'Inglés', 'Otros idiomas',
       'Euskera', 'Valenciano', 'Francés', 'Árabe', 'Alemán', 'Italiano',
       'Portugués', 'Ruso', 'Chino', 'Rumano'], dtype=object)

In [40]:
# Inspección: primer idioma extranjero (cuatro idiomas extranjeros: 'idioma1', 'idioma2', 'idioma3', 'idioma4'): 
df['idioma1_'].unique()

array(['Inglés', nan, 'Catalán', 'Francés', 'Italiano', 'Euskera',
       'Alemán', 'Español', 'Gallego', 'Portugués', 'Otros idiomas',
       'Valenciano', 'Árabe', 'Chino', 'Ruso', 'Rumano'], dtype=object)

In [41]:
# Inspección: nivel de conocimiento del idioma declarado por el encuestado: 
df['idioma1_nivel_'].unique()

array(['Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)',
       'Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)',
       nan, 'Nivel bajo (Entiende y usa las expresiones más comunes )'],
      dtype=object)

In [42]:
# Inspección: nivel de conocimiento del idioma acreditado: 
df['idioma1_acred_'].unique()

array(['B2 (FCE, DELF B2, ZDfB, …)', 'B1 (PET, DELF B1, ZD, …)',
       'C1 (CAE, DALF C1, PWD, …)', nan,
       'A2 (DELF A2, Fit in Deutsch 2, …)', 'C2 (CPE, DALF C2, ZOP, …)',
       'A1 (DELF A1, Fit in Deutsch 1, ….)'], dtype=object)

#### Idiomas maternos:

In [43]:
# Auxiliares para comprobaciones (conserva las variables que se transforman con pd.get_dummies): 
df['materno_1']=df['idioma_m1_']
df['materno_2']=df['idioma_m2_']

In [44]:
# Columnas binarias para idiomas maternos: 
df=pd.get_dummies(data=df,columns=['idioma_m1_','idioma_m2_'],prefix=['m1','m2'])

In [45]:
# Se prenscinde de las columnas referidas a 'Otros_idiomas'
del df['m1_Otros idiomas'], df['m2_Otros idiomas']

In [46]:
# Inspección: 
df.columns[-(df['idioma_m1'].nunique()-1)*2:]

Index(['m1_Alemán', 'm1_Catalán', 'm1_Chino', 'm1_Español', 'm1_Euskera',
       'm1_Francés', 'm1_Gallego', 'm1_Inglés', 'm1_Italiano', 'm1_Portugués',
       'm1_Rumano', 'm1_Ruso', 'm1_Valenciano', 'm1_Árabe', 'm2_Alemán',
       'm2_Catalán', 'm2_Chino', 'm2_Español', 'm2_Euskera', 'm2_Francés',
       'm2_Gallego', 'm2_Inglés', 'm2_Italiano', 'm2_Portugués', 'm2_Rumano',
       'm2_Ruso', 'm2_Valenciano', 'm2_Árabe'],
      dtype='object')

In [47]:
# lista de columnas de idioma materno (aux.):
m1_m2=list(filter(lambda col:col.startswith('m1_') or col.startswith('m2_'),df.columns))
#m1_m2

In [48]:
# Lista de idiomas registrados en columnas binarias: 
idiomas=list(set(map(lambda string: string.split('_')[1],m1_m2)))
idiomas

['Euskera',
 'Rumano',
 'Portugués',
 'Francés',
 'Árabe',
 'Italiano',
 'Gallego',
 'Inglés',
 'Valenciano',
 'Español',
 'Alemán',
 'Catalán',
 'Ruso',
 'Chino']

In [49]:
# Columnas binarias para cada idioma materno: reune las dos columnas para idiomas maternos m1_idioma y m2_idioma.
maternos=list()
for idioma in idiomas:
    df[str(idioma)+'_materno']=df['m1_'+str(idioma)]+df['m2_'+str(idioma)]
    maternos.append(str(idioma)+'_materno')
df[maternos].agg(['unique'])

,Euskera_materno,Rumano_materno,Portugués_materno,Francés_materno,Árabe_materno,Italiano_materno,Gallego_materno,Inglés_materno,Valenciano_materno,Español_materno,Alemán_materno,Catalán_materno,Ruso_materno,Chino_materno
unique,"[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[1, 0]","[0, 1]","[0, 1]","[0, 1]","[0, 1]"


In [50]:
# Comprobación:
df[maternos].sum(), df['materno_1'].value_counts()+df['materno_2'].value_counts() 

(Euskera_materno        1791
 Rumano_materno           52
 Portugués_materno        71
 Francés_materno         185
 Árabe_materno            58
 Italiano_materno         64
 Gallego_materno        2712
 Inglés_materno          484
 Valenciano_materno     2224
 Español_materno       41303
 Alemán_materno           79
 Catalán_materno        5487
 Ruso_materno             44
 Chino_materno            31
 dtype: int64,
 Alemán              79
 Catalán           5487
 Chino               31
 Español          41303
 Euskera           1791
 Francés            185
 Gallego           2712
 Inglés             484
 Italiano            64
 Otros idiomas      399
 Portugués           71
 Rumano              52
 Ruso                44
 Valenciano        2224
 Árabe               58
 dtype: int64)

#### Idiomas extranjeros: 

In [51]:
# Una colección de dummies para cada idioma extranjero (idioma1, idioma2, idioma3, idioma4):
df=pd.get_dummies(data=df,columns=['idioma1_','idioma2_','idioma3_','idioma4_'],prefix=['i1','i2','i3','i4'])

In [52]:
# Se prescinde de columnas referidas a 'Otros_idiomas'
for n in range(1,5):
    del df['i'+str(n)+'_Otros idiomas']

In [53]:
# Inspección: 
df.columns[-(df['idioma_m1'].nunique()-1)*4:]

Index(['i1_Alemán', 'i1_Catalán', 'i1_Chino', 'i1_Español', 'i1_Euskera',
       'i1_Francés', 'i1_Gallego', 'i1_Inglés', 'i1_Italiano', 'i1_Portugués',
       'i1_Rumano', 'i1_Ruso', 'i1_Valenciano', 'i1_Árabe', 'i2_Alemán',
       'i2_Catalán', 'i2_Chino', 'i2_Español', 'i2_Euskera', 'i2_Francés',
       'i2_Gallego', 'i2_Inglés', 'i2_Italiano', 'i2_Portugués', 'i2_Rumano',
       'i2_Ruso', 'i2_Valenciano', 'i2_Árabe', 'i3_Alemán', 'i3_Catalán',
       'i3_Chino', 'i3_Español', 'i3_Euskera', 'i3_Francés', 'i3_Gallego',
       'i3_Inglés', 'i3_Italiano', 'i3_Portugués', 'i3_Rumano', 'i3_Ruso',
       'i3_Valenciano', 'i3_Árabe', 'i4_Alemán', 'i4_Catalán', 'i4_Chino',
       'i4_Español', 'i4_Euskera', 'i4_Francés', 'i4_Gallego', 'i4_Inglés',
       'i4_Italiano', 'i4_Portugués', 'i4_Rumano', 'i4_Ruso', 'i4_Valenciano',
       'i4_Árabe'],
      dtype='object')

In [54]:
# listas_idiomas=lista de listas de columnas dummy para cada idioma extranjero:
listas_idiomas=[list(filter(lambda col:col.startswith('i'+str(n)+'_'),df.columns)) for n in range(1,5)]
#listas_idiomas

In [55]:
# Inspección: lista de primeros idiomas extranjeros (i1_idioma) y valores únicos en col. binarias.
listas_idiomas[0], df[listas_idiomas[0]].agg(['unique'])

(['i1_Alemán',
  'i1_Catalán',
  'i1_Chino',
  'i1_Español',
  'i1_Euskera',
  'i1_Francés',
  'i1_Gallego',
  'i1_Inglés',
  'i1_Italiano',
  'i1_Portugués',
  'i1_Rumano',
  'i1_Ruso',
  'i1_Valenciano',
  'i1_Árabe'],
        i1_Alemán i1_Catalán i1_Chino i1_Español i1_Euskera i1_Francés  \
 unique    [0, 1]     [0, 1]   [0, 1]     [0, 1]     [0, 1]     [0, 1]   
 
        i1_Gallego i1_Inglés i1_Italiano i1_Portugués i1_Rumano i1_Ruso  \
 unique     [0, 1]    [1, 0]      [0, 1]       [0, 1]    [0, 1]  [0, 1]   
 
        i1_Valenciano i1_Árabe  
 unique        [0, 1]   [0, 1]  )

In [56]:
# Convierte listas_idiomas (lista de listas) en una lista única de idiomas (sin agrupar en sublistas)
lista_idiomas_unica=list(reduce(lambda lista1,lista2: lista1+lista2, listas_idiomas))

In [57]:
df[lista_idiomas_unica].agg(['value_counts'])

,i1_Alemán,i1_Catalán,i1_Chino,i1_Español,i1_Euskera,i1_Francés,i1_Gallego,i1_Inglés,i1_Italiano,i1_Portugués,i1_Rumano,i1_Ruso,i1_Valenciano,i1_Árabe,i2_Alemán,i2_Catalán,i2_Chino,i2_Español,i2_Euskera,i2_Francés,i2_Gallego,i2_Inglés,i2_Italiano,i2_Portugués,i2_Rumano,i2_Ruso,i2_Valenciano,i2_Árabe,i3_Alemán,i3_Catalán,i3_Chino,i3_Español,i3_Euskera,i3_Francés,i3_Gallego,i3_Inglés,i3_Italiano,i3_Portugués,i3_Rumano,i3_Ruso,i3_Valenciano,i3_Árabe,i4_Alemán,i4_Catalán,i4_Chino,i4_Español,i4_Euskera,i4_Francés,i4_Gallego,i4_Inglés,i4_Italiano,i4_Portugués,i4_Rumano,i4_Ruso,i4_Valenciano,i4_Árabe
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,42902,42140,43123,41598,42568,41546,42908,9518,42627,42843,43128,43121,42454,43119,41412,42539,43056,42981,42825,35803,42973,38134,41320,42080,43119,43087,42778,43101,42266,42906,43050,43109,43056,41816,43070,42704,42251,42672,43123,43076,43039,43077,42974,43068,43096,43126,43121,42959,43118,43094,42901,43020,43130,43101,43118,43102
1,232,994,11,1536,566,1588,226,33616,507,291,6,13,680,15,1722,595,78,153,309,7331,161,5000,1814,1054,15,47,356,33,868,228,84,25,78,1318,64,430,883,462,11,58,95,57,160,66,38,8,13,175,16,40,233,114,4,33,16,32


### Nuevas variables: 'i', con i={idiomas observados}
Una columna binaria por idioma, con valor 1 cuando el encuestado conoce el idioma, ya sea como lengua materna o como idioma extranjero aprendido.   
  
No tiene en cuenta el nivel de conocimiento del idioma ni su acreditación oficial. Por tanto, tiene valores informados siempre que el encuestado menciona el idioma, incluso cuando no informa de su nivel de conocimiento o de acreditación. 

In [58]:
# Se reunen todas las columnas referidas al mismo idioma, sea como lengua materna o como lengua extranjera: 
for idioma in idiomas: 
    df[str(idioma)]=df[str(idioma)+'_materno']
    for n in range(1,5): 
        df[str(idioma)]=df[str(idioma)]+df['i'+str(n)+'_'+str(idioma)] 

In [59]:
# Inspección: 
df[[str(idioma) for idioma in idiomas]].agg(['value_counts'])

,Euskera,Rumano,Portugués,Francés,Árabe,Italiano,Gallego,Inglés,Valenciano,Español,Alemán,Catalán,Ruso,Chino
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,40377,43046,41142,32537,42939,39633,39955,3564,39763,109,40073,35764,42939,42892
1,2757,88,1992,10597,195,3501,3179,39570,3371,43025,3061,7370,195,242


### Contador de repeticiones
Se construye un **contador de repeticiones de valores iguales en una selección de columnas de un dataframe**.  
  
Permite conocer, para cada valor presente en la selección, cuantas observaciones tienen ese valor repetido en las distintas columnas para cada número de repeticiones concebible.  
   
Aquí se emplea para comprobar que el resultado cuadra con los registros resultantes de la tranformación de las columnas asociadas a los idiomas. 

In [60]:
def une_listas(L1,L2):  
    l1=list(set(L1))
    return l1+[x for x in L2 if x not in l1]

In [61]:
def formato(valor):
    try: valor=int(valor)
    except: pass
    return valor

In [62]:
def cuenta_repeticiones(data,columns=None): 
    from functools import reduce
    if columns==None: 
        columns=data.columns
    df_aux=data[columns]
    cols=df_aux.columns
    valores_brutos=list(map(lambda col: df_aux[col].unique().tolist(),cols))
    valores=list(reduce(une_listas,valores_brutos))
    valores_notna=[v for v in valores if v==v] # prescinde de nan
    valores_notna=list(map(formato,valores_notna))
    D_out=dict(map(lambda v: (v,df_aux[df_aux==v].count(axis=1)),valores_notna))
    if len(valores)!=len(valores_notna):
        D_out['nan']=len(cols)-df_aux[cols].count(axis=1)
    dx=pd.DataFrame(D_out)
    N=range(len(cols)+1)
    D=dict(map(lambda v: (v,list(map(lambda n: dx[v][dx[v]==n].count(),N))),dx.columns))
    out=pd.DataFrame(D)
    out.index.name='Repetitions'
    return out

In [63]:
check=cuenta_repeticiones(df,['idioma_m1','idioma_m2','idioma1','idioma2','idioma3','idioma4'])
check.rename(columns=D_dict['idioma1'])

,Español,Catalán,Euskera,Gallego,Valenciano,Inglés,Francés,Alemán,Italiano,Portugués,Rumano,Árabe,Chino,Ruso,Otros idiomas,nan
Repetitions,,,,,,,,,,,,,,,,
0,109,35764,40377,39955,39763,3564,32537,40073,39633,41142,43046,42939,42892,42939,41844,315
1,43025,7370,2757,3179,3371,39570,10597,3061,3501,1992,88,195,242,195,1200,1787
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,83,6564
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,16550
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,15882
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2036
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [64]:
# Comprobación: los resultados cuadran con los calculados mediante cuenta_repeticiones:  
df[[str(idioma) for idioma in idiomas]].agg(['value_counts'])

,Euskera,Rumano,Portugués,Francés,Árabe,Italiano,Gallego,Inglés,Valenciano,Español,Alemán,Catalán,Ruso,Chino
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,40377,43046,41142,32537,42939,39633,39955,3564,39763,109,40073,35764,42939,42892
1,2757,88,1992,10597,195,3501,3179,39570,3371,43025,3061,7370,195,242


In [65]:
# Diccionarios asociados a las variables 'i' (i={idiomas observados})
for idioma in idiomas: 
    D_describe[str(idioma)]='Conoce o no el idioma (binaria)'
    D_dict[str(idioma)]=dict()
    D_dict[str(idioma)][0]='No'
    D_dict[str(idioma)][1]='Sí'

In [66]:
# Variables literales para 'i' (i={idiomas}):
for idioma in idiomas: 
    df[str(idioma)+'_']=df[str(idioma)].replace(D_dict[str(idioma)])
    print (idioma, df[str(idioma)+'_'].unique())

Euskera ['No' 'Sí']
Rumano ['No' 'Sí']
Portugués ['No' 'Sí']
Francés ['No' 'Sí']
Árabe ['No' 'Sí']
Italiano ['No' 'Sí']
Gallego ['No' 'Sí']
Inglés ['Sí' 'No']
Valenciano ['No' 'Sí']
Español ['Sí' 'No']
Alemán ['No' 'Sí']
Catalán ['No' 'Sí']
Ruso ['No' 'Sí']
Chino ['No' 'Sí']


### Nuevas variables: 'i_nivel', con i={idiomas observados}
Una columna por idioma, graduada de 0 a 3 según el nivel de conocimiento declarado por el encuestado. Se imputa nivel alto (valor 3) a los hablantes nativos del idioma.  
  
No se tienen en cuenta los registros para los que el nivel de conocimiento del idioma no está informado. Por ello, tiene menos valores informados que las variables de idioma 'i' (i={idiomas}).
  
**Nota**: **Se invierte el sentido de la escala** respecto a la definida originalmente. Esto es, se asocian los niveles más altos de conocimiento del idioma a valores numéricos más altos y se asocia el valor 0 a no conocer el idioma. 

In [67]:
# Nuevas variables de idiomas graduados por nivel 
df['idioma1_nivel_'].unique() # Inspección

array(['Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)',
       'Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)',
       nan, 'Nivel bajo (Entiende y usa las expresiones más comunes )'],
      dtype=object)

In [68]:
# Escala numérica orginal (se invierte en las escalas definidas para las variables nuevas)
leyenda('idioma1_nivel')

1 	 Nivel alto (Entiende una gran variedad de textos complicados y usa el idioma con flexibilidad)
2 	 Nivel medio (Entiende lo esencial en el lenguaje común y redacta textos sencillos)
3 	 Nivel bajo (Entiende y usa las expresiones más comunes )
9 	 NS/NC
  	 No aplicable


In [69]:
# Se prescinde de los registros de idiomas  para los que el nivel de conocimiento no está informado:
for i in range(1,5):
    infs_idioma=df['idioma'+str(i)].count()
    infs_nivel=df['idioma'+str(i)+'_nivel'].count()
    dif=infs_idioma-infs_nivel
    print (f'idioma {i}: Informados idioma: {infs_idioma}, informados nivel: {infs_nivel}, omitidos: {dif}')

idioma 1: Informados idioma: 40371, informados nivel: 40047, omitidos: 324
idioma 2: Informados idioma: 19096, informados nivel: 18967, omitidos: 129
idioma 3: Informados idioma: 4987, informados nivel: 4949, omitidos: 38
idioma 4: Informados idioma: 1093, informados nivel: 1086, omitidos: 7


In [70]:
# Inspección: lista de primeros idiomas extranjeros (i1_idioma), valores únicos en col. binarias y nivel.
listas_idiomas[0], df[listas_idiomas[0]].agg(['unique']), df['idioma'+str(1)+'_nivel'].unique()

(['i1_Alemán',
  'i1_Catalán',
  'i1_Chino',
  'i1_Español',
  'i1_Euskera',
  'i1_Francés',
  'i1_Gallego',
  'i1_Inglés',
  'i1_Italiano',
  'i1_Portugués',
  'i1_Rumano',
  'i1_Ruso',
  'i1_Valenciano',
  'i1_Árabe'],
        i1_Alemán i1_Catalán i1_Chino i1_Español i1_Euskera i1_Francés  \
 unique    [0, 1]     [0, 1]   [0, 1]     [0, 1]     [0, 1]     [0, 1]   
 
        i1_Gallego i1_Inglés i1_Italiano i1_Portugués i1_Rumano i1_Ruso  \
 unique     [0, 1]    [1, 0]      [0, 1]       [0, 1]    [0, 1]  [0, 1]   
 
        i1_Valenciano i1_Árabe  
 unique        [0, 1]   [0, 1]  ,
 array([ 2.,  1., nan,  3.]))

#### Para los idiomas maternos:

In [71]:
# Para los idiomas maternos: se imputa nivel alto (3) 
for materno in maternos: 
    df[materno]=df[materno]*3
df[maternos].agg(['value_counts'])

,Euskera_materno,Rumano_materno,Portugués_materno,Francés_materno,Árabe_materno,Italiano_materno,Gallego_materno,Inglés_materno,Valenciano_materno,Español_materno,Alemán_materno,Catalán_materno,Ruso_materno,Chino_materno
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,41343,43082,43063,42949,43076,43070,40422,42650,40910,1831,43055,37647,43090,43103
3,1791,52,71,185,58,64,2712,484,2224,41303,79,5487,44,31


#### Para los idiomas extranjeros:

In [72]:
# Variables de nivel auxiliares para invertir el sentido de la escala: 
for n in range(1,5):
    df['nivel_aux_i'+str(n)]=df['idioma'+str(n)+'_nivel'].replace({1:3,3:1,None:0})
df[['idioma1_nivel','nivel_aux_i1']].groupby('idioma1_nivel').agg(['unique']) # Comprobación: 

,nivel_aux_i1
,unique
idioma1_nivel,
1.0,[3.0]
2.0,[2.0]
3.0,[1.0]


In [73]:
# Nivel declarado de conocimiento del idioma: 
# Se multiplican las variables dummy por el nivel de conocimento declarado de cada idioma extranjero. 
for n in range(4):
    for idioma in listas_idiomas[n]:
        df[idioma+'_nivel']=df[idioma]*df['nivel_aux_i'+str(n+1)] # nivel de acuerdo con la escala redefinida

In [74]:
# lista_idiomas_nivel =lista de variables referidas al nivel de cada idioma
lista_idiomas_nivel=list(map(lambda col: col+'_nivel',lista_idiomas_unica))
df[lista_idiomas_nivel].agg(['value_counts']) # Inspección

,i1_Alemán_nivel,i1_Catalán_nivel,i1_Chino_nivel,i1_Español_nivel,i1_Euskera_nivel,i1_Francés_nivel,i1_Gallego_nivel,i1_Inglés_nivel,i1_Italiano_nivel,i1_Portugués_nivel,i1_Rumano_nivel,i1_Ruso_nivel,i1_Valenciano_nivel,i1_Árabe_nivel,i2_Alemán_nivel,i2_Catalán_nivel,i2_Chino_nivel,i2_Español_nivel,i2_Euskera_nivel,i2_Francés_nivel,i2_Gallego_nivel,i2_Inglés_nivel,i2_Italiano_nivel,i2_Portugués_nivel,i2_Rumano_nivel,i2_Ruso_nivel,i2_Valenciano_nivel,i2_Árabe_nivel,i3_Alemán_nivel,i3_Catalán_nivel,i3_Chino_nivel,i3_Español_nivel,i3_Euskera_nivel,i3_Francés_nivel,i3_Gallego_nivel,i3_Inglés_nivel,i3_Italiano_nivel,i3_Portugués_nivel,i3_Rumano_nivel,i3_Ruso_nivel,i3_Valenciano_nivel,i3_Árabe_nivel,i4_Alemán_nivel,i4_Catalán_nivel,i4_Chino_nivel,i4_Español_nivel,i4_Euskera_nivel,i4_Francés_nivel,i4_Gallego_nivel,i4_Inglés_nivel,i4_Italiano_nivel,i4_Portugués_nivel,i4_Rumano_nivel,i4_Ruso_nivel,i4_Valenciano_nivel,i4_Árabe_nivel
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0.0,42902,42148,43123,41620,42574,41556,42913,9774,42630,42847,43128.0,43121,42463,43119,41423,42544,43058,42982.0,42826,35843,42973,38182,41333,42084,43119,43087,42781,43101,42276,42906,43050,43109.0,43056,41826,43070,42708,42257,42674,43123,43077,43040,43077,42975,43069,43096.0,43126.0,43121,42959,43118,43094,42904,43020,43130,43101,43118,43102
1.0,43,39,6,5,49,338,14,2161,55,33,NaN,3,33,1,677,92,44,NaN,100,2925,36,431,439,291,3,19,50,20,492,52,63,NaN,37,594,18,48,301,148,6,42,25,33,99,19,28.0,1.0,8,92,4,3,104,45,2,19,6,23
2.0,53,153,3,21,152,601,51,15192,183,101,1.0,2,216,8,739,218,28,5.0,135,3136,53,2426,836,478,6,17,139,8,291,100,18,1.0,27,563,27,217,414,216,4,13,33,20,45,29,10.0,NaN,2,70,4,15,97,54,1,11,4,8
3.0,136,794,2,1488,359,639,156,16007,266,153,5.0,8,422,6,295,280,4,147.0,73,1230,72,2095,526,281,6,11,164,5,75,76,3,24.0,14,151,19,161,162,96,1,2,36,4,15,17,NaN,7.0,3,13,8,22,29,15,1,3,6,1


In [75]:
# Se reunen todas las columnas referidas al mismo idioma: 
for idioma in idiomas: 
    df[str(idioma)+'_nivel']=df[str(idioma)+'_materno']
    for n in range(1,5): 
        df[str(idioma)+'_nivel']=df[str(idioma)+'_nivel']+df['i'+str(n)+'_'+str(idioma)+'_nivel']  

In [76]:
# Inspección: 
df[[str(idioma)+'_nivel' for idioma in idiomas]].agg(['value_counts'])

,Euskera_nivel,Rumano_nivel,Portugués_nivel,Francés_nivel,Árabe_nivel,Italiano_nivel,Gallego_nivel,Inglés_nivel,Valenciano_nivel,Español_nivel,Alemán_nivel,Catalán_nivel,Ruso_nivel,Chino_nivel
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0.0,40384,43046,41152,32597,42939,39658,39960,3872,39776,132,40095,35778,42940,42894
1.0,194,11,517,3949,77,899,72,2643,114,6,1311,202,83,141
2.0,316,12,849,4370,44,1530,135,17850,392,27,1128,500,43,59
3.0,2240,65,616,2218,74,1047,2967,18769,2852,42969,600,6654,68,40


In [77]:
# Cálculos para comprobaciones (para cada idioma, con su código numérico como valor de n):
n=4 # (4=gallego)
# n3= núm. registros idioma n con nivel 3 (escala modificada):
n3=df['idioma_m1'][df['idioma_m1']==n].count()+df['idioma_m2'][df['idioma_m2']==n].count()\
  +df['idioma1'][(df['idioma1']==n) & (df['idioma1_nivel']==1)].count()\
  +df['idioma2'][(df['idioma2']==n) & (df['idioma2_nivel']==1)].count()\
  +df['idioma3'][(df['idioma3']==n) & (df['idioma3_nivel']==1)].count()\
  +df['idioma4'][(df['idioma4']==n) & (df['idioma4_nivel']==1)].count() 
# n2= núm. registros idioma n con nivel 2:
n2=df['idioma1'][(df['idioma1']==n) & (df['idioma1_nivel']==2)].count()\
  +df['idioma2'][(df['idioma2']==n) & (df['idioma2_nivel']==2)].count()\
  +df['idioma3'][(df['idioma3']==n) & (df['idioma3_nivel']==2)].count()\
  +df['idioma4'][(df['idioma4']==n) & (df['idioma4_nivel']==2)].count()
# n1= núm. registros idioma n con nivel 1 (escala modificada):
n1=df['idioma1'][(df['idioma1']==n) & (df['idioma1_nivel']==3)].count()\
  +df['idioma2'][(df['idioma2']==n) & (df['idioma2_nivel']==3)].count()\
  +df['idioma3'][(df['idioma3']==n) & (df['idioma3_nivel']==3)].count()\
  +df['idioma4'][(df['idioma4']==n) & (df['idioma4_nivel']==3)].count()
# count funciona bien como agregador porque tenemos la garantía de que ninguna observación menciona el mismo ...
# ... idioma en dos columnas (de idioma materno o extranjero) distintas. 
n1,n2,n3

(72, 135, 2967)

In [78]:
# Diccionarios asociados a las variables 'i_nivel' (i={idiomas registrados})
for idioma in idiomas: 
    D_describe[str(idioma)+'_nivel']='Nivel de conocimiento del idioma declarado por el encuestado'
    D_dict[str(idioma)+'_nivel']=dict()
    D_dict[str(idioma)+'_nivel'][0]='Desconocido' # No conoce el idioma o el nivel no está informado
    D_dict[str(idioma)+'_nivel'][1]='Nivel bajo'
    D_dict[str(idioma)+'_nivel'][2]='Nivel medio'
    D_dict[str(idioma)+'_nivel'][3]='Nivel alto'

In [79]:
# Variables literales para 'i_nivel' (i={idiomas}):
for idioma in idiomas: 
    df[str(idioma)+'_nivel_']=df[str(idioma)+'_nivel'].replace(D_dict[str(idioma)+'_nivel'])
    print (idioma, df[str(idioma)+'_nivel_'].unique())

Euskera ['Desconocido' 'Nivel medio' 'Nivel alto' 'Nivel bajo']
Rumano ['Desconocido' 'Nivel alto' 'Nivel medio' 'Nivel bajo']
Portugués ['Desconocido' 'Nivel medio' 'Nivel alto' 'Nivel bajo']
Francés ['Desconocido' 'Nivel medio' 'Nivel bajo' 'Nivel alto']
Árabe ['Desconocido' 'Nivel medio' 'Nivel alto' 'Nivel bajo']
Italiano ['Desconocido' 'Nivel medio' 'Nivel alto' 'Nivel bajo']
Gallego ['Desconocido' 'Nivel alto' 'Nivel medio' 'Nivel bajo']
Inglés ['Nivel medio' 'Nivel alto' 'Desconocido' 'Nivel bajo']
Valenciano ['Desconocido' 'Nivel medio' 'Nivel alto' 'Nivel bajo']
Español ['Nivel alto' 'Desconocido' 'Nivel medio' 'Nivel bajo']
Alemán ['Desconocido' 'Nivel bajo' 'Nivel medio' 'Nivel alto']
Catalán ['Desconocido' 'Nivel alto' 'Nivel medio' 'Nivel bajo']
Ruso ['Desconocido' 'Nivel bajo' 'Nivel medio' 'Nivel alto']
Chino ['Desconocido' 'Nivel bajo' 'Nivel medio' 'Nivel alto']


### Nuevas variables: 'i_acred', con i={idiomas observados}
Una columna por idioma, graduada de 0 a 6 según el nivel de conocimiento acreditado con título oficial.  
  
No se tienen en cuenta los registros para los que el nivel de acreditación de conocimiento del idioma no está informado. Por ello, tiene menos valores informados que las variables de idioma 'i' (i={idiomas}).   
  
Se imputa nivel C1 a los hablantes nativos del idioma.

In [80]:
leyenda('idioma1_acred')

1 	 A1 (DELF A1, Fit in Deutsch 1, ….)
2 	 A2 (DELF A2, Fit in Deutsch 2, …)
3 	 B1 (PET, DELF B1, ZD, …)
9 	 NS/NC
  	 No_aplicable
4 	 B2 (FCE, DELF B2, ZDfB, …)
5 	 C1 (CAE, DALF C1, PWD, …)
6 	 C2 (CPE, DALF C2, ZOP, …)


In [81]:
df['idioma1_acred_'].unique() # Inspección

array(['B2 (FCE, DELF B2, ZDfB, …)', 'B1 (PET, DELF B1, ZD, …)',
       'C1 (CAE, DALF C1, PWD, …)', nan,
       'A2 (DELF A2, Fit in Deutsch 2, …)', 'C2 (CPE, DALF C2, ZOP, …)',
       'A1 (DELF A1, Fit in Deutsch 1, ….)'], dtype=object)

In [82]:
# Se abrevian los registros literales de las variables referidas al nivel de acreditación: 
for n in range(1,5):
    df['idioma'+str(n)+'_acred_']=df['idioma'+str(n)+'_acred_'].str.split(' ',1).str[0]
df['idioma3_acred_'].unique() # Inspección

array([nan, 'B2', 'B1', 'A2', 'C1', 'A1', 'C2'], dtype=object)

In [83]:
# Se prescinde de los registros de idiomas para los que la acreditación no está informada:
for i in range(1,5):
    infs_idioma=df['idioma'+str(i)].count()
    infs_nivel=df['idioma'+str(i)+'_acred'].count()
    dif=infs_idioma-infs_nivel
    print (f'idioma {i}: Informados idioma: {infs_idioma}, informados nivel: {infs_nivel}, omitidos: {dif}')

idioma 1: Informados idioma: 40371, informados nivel: 23494, omitidos: 16877
idioma 2: Informados idioma: 19096, informados nivel: 7707, omitidos: 11389
idioma 3: Informados idioma: 4987, informados nivel: 1733, omitidos: 3254
idioma 4: Informados idioma: 1093, informados nivel: 355, omitidos: 738


#### Para los idiomas maternos:

In [84]:
# Para los idiomas maternos: se imputa nivel C1 (valor 5)
for materno in maternos: 
    df[materno]=df[materno]*5/3 # (deshace la imputación por nivel (valor 3) y asigna acreditación C1 (valor 5))
df[maternos].agg(['value_counts'])

,Euskera_materno,Rumano_materno,Portugués_materno,Francés_materno,Árabe_materno,Italiano_materno,Gallego_materno,Inglés_materno,Valenciano_materno,Español_materno,Alemán_materno,Catalán_materno,Ruso_materno,Chino_materno
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0.0,41343,43082,43063,42949,43076,43070,40422,42650,40910,1831,43055,37647,43090,43103
5.0,1791,52,71,185,58,64,2712,484,2224,41303,79,5487,44,31


#### Para los idiomas extranjeros:

In [85]:
df[lista_idiomas_unica].agg(['value_counts'])

,i1_Alemán,i1_Catalán,i1_Chino,i1_Español,i1_Euskera,i1_Francés,i1_Gallego,i1_Inglés,i1_Italiano,i1_Portugués,i1_Rumano,i1_Ruso,i1_Valenciano,i1_Árabe,i2_Alemán,i2_Catalán,i2_Chino,i2_Español,i2_Euskera,i2_Francés,i2_Gallego,i2_Inglés,i2_Italiano,i2_Portugués,i2_Rumano,i2_Ruso,i2_Valenciano,i2_Árabe,i3_Alemán,i3_Catalán,i3_Chino,i3_Español,i3_Euskera,i3_Francés,i3_Gallego,i3_Inglés,i3_Italiano,i3_Portugués,i3_Rumano,i3_Ruso,i3_Valenciano,i3_Árabe,i4_Alemán,i4_Catalán,i4_Chino,i4_Español,i4_Euskera,i4_Francés,i4_Gallego,i4_Inglés,i4_Italiano,i4_Portugués,i4_Rumano,i4_Ruso,i4_Valenciano,i4_Árabe
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,42902,42140,43123,41598,42568,41546,42908,9518,42627,42843,43128,43121,42454,43119,41412,42539,43056,42981,42825,35803,42973,38134,41320,42080,43119,43087,42778,43101,42266,42906,43050,43109,43056,41816,43070,42704,42251,42672,43123,43076,43039,43077,42974,43068,43096,43126,43121,42959,43118,43094,42901,43020,43130,43101,43118,43102
1,232,994,11,1536,566,1588,226,33616,507,291,6,13,680,15,1722,595,78,153,309,7331,161,5000,1814,1054,15,47,356,33,868,228,84,25,78,1318,64,430,883,462,11,58,95,57,160,66,38,8,13,175,16,40,233,114,4,33,16,32


In [86]:
# Nivel acreditado del idioma: 
# Se multiplican las variables dummy por el nivel de acreditación de cada idioma extranjero. 
for n in range(4):
    for idioma in listas_idiomas[n]:
        df[idioma]=df[idioma]*df['idioma'+str(n+1)+'_acred'].mask(df['idioma'+str(n+1)+'_acred'].isna(),0)
df[lista_idiomas_unica].agg(['value_counts']) # Inspección

,i1_Alemán,i1_Catalán,i1_Chino,i1_Español,i1_Euskera,i1_Francés,i1_Gallego,i1_Inglés,i1_Italiano,i1_Portugués,i1_Rumano,i1_Ruso,i1_Valenciano,i1_Árabe,i2_Alemán,i2_Catalán,i2_Chino,i2_Español,i2_Euskera,i2_Francés,i2_Gallego,i2_Inglés,i2_Italiano,i2_Portugués,i2_Rumano,i2_Ruso,i2_Valenciano,i2_Árabe,i3_Alemán,i3_Catalán,i3_Chino,i3_Español,i3_Euskera,i3_Francés,i3_Gallego,i3_Inglés,i3_Italiano,i3_Portugués,i3_Rumano,i3_Ruso,i3_Valenciano,i3_Árabe,i4_Alemán,i4_Catalán,i4_Chino,i4_Español,i4_Euskera,i4_Francés,i4_Gallego,i4_Inglés,i4_Italiano,i4_Portugués,i4_Rumano,i4_Ruso,i4_Valenciano,i4_Árabe
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0.0,42974,42548,43125.0,42742,42748,42424,43032,22782,42900,43010,43132.0,43130.0,42737,43127.0,42216,42926,43087,43090.0,42987,40935,43103,40268,42536,42865,43131.0,43117.0,42960,43127.0,42712,43073,43094.0,43127.0,43108.0,42725,43125.0,42889,42909,43046,43133.0,43115.0,43092.0,43107,43057,43120.0,43111.0,43133.0,43129.0,43081.0,43132.0,43106.0,43079,43115.0,43133.0,43127.0,43127.0,43120.0
1.0,9,9,4.0,13,2,19,4,147,6,4,NaN,NaN,18,NaN,87,9,6,4.0,5,121,1,23,34,11,NaN,NaN,4,NaN,59,5,7.0,NaN,3.0,40,NaN,1,18,12,NaN,5.0,NaN,5,12,NaN,6.0,NaN,1.0,4.0,NaN,NaN,8,NaN,NaN,2.0,NaN,3.0
2.0,11,16,1.0,8,10,47,1,413,22,6,NaN,NaN,29,NaN,183,14,13,NaN,13,364,3,72,86,55,1.0,5.0,15,2.0,120,3,19.0,NaN,4.0,89,NaN,6,37,14,1.0,6.0,2.0,8,23,3.0,3.0,NaN,1.0,13.0,NaN,NaN,9,5.0,NaN,2.0,1.0,6.0
3.0,24,30,1.0,16,49,155,11,5488,68,39,NaN,NaN,60,4.0,286,26,16,3.0,40,641,1,732,193,88,NaN,6.0,34,2.0,123,11,6.0,1.0,6.0,116,1.0,72,79,23,NaN,7.0,11.0,7,19,1.0,11.0,NaN,1.0,18.0,NaN,5.0,14,5.0,NaN,2.0,NaN,2.0
4.0,48,60,1.0,34,112,250,15,8025,83,39,NaN,NaN,46,NaN,227,44,8,1.0,47,711,7,1125,191,68,1.0,4.0,31,1.0,83,12,6.0,1.0,7.0,118,2.0,101,56,25,NaN,1.0,7.0,2,14,2.0,2.0,NaN,NaN,13.0,1.0,13.0,16,7.0,1.0,1.0,3.0,3.0
5.0,53,335,2.0,134,181,150,39,5164,42,20,1.0,2.0,194,1.0,107,85,3,14.0,38,283,13,723,80,40,1.0,1.0,75,2.0,29,22,2.0,3.0,6.0,31,4.0,50,21,9,NaN,NaN,16.0,2,7,6.0,NaN,NaN,1.0,5.0,1.0,8.0,6,2.0,NaN,NaN,2.0,NaN
6.0,15,136,NaN,187,32,89,32,1115,13,16,1.0,2.0,50,2.0,28,30,1,22.0,4,79,6,191,14,7,NaN,1.0,15,NaN,8,8,NaN,2.0,NaN,15,2.0,15,14,5,NaN,NaN,6.0,3,2,2.0,1.0,1.0,1.0,NaN,NaN,2.0,2,NaN,NaN,NaN,1.0,NaN


In [87]:
# Se reunen todas las columnas referidas al mismo idioma: 
for idioma in idiomas: 
    df[str(idioma)+'_acred']=df[str(idioma)+'_materno']
    for n in range(1,5): 
        df[str(idioma)+'_acred']=df[str(idioma)+'_acred']+df['i'+str(n)+'_'+str(idioma)]

In [88]:
# Comprobación: 
df[[str(idioma)+'_acred' for idioma in idiomas]].agg(['value_counts'])

,Euskera_acred,Rumano_acred,Portugués_acred,Francés_acred,Árabe_acred,Italiano_acred,Gallego_acred,Inglés_acred,Valenciano_acred,Español_acred,Alemán_acred,Catalán_acred,Ruso_acred,Chino_acred
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0.0,40779,43075.0,42563,39578,43021,41958,40278,19159,40290,1387,41478,36778,43043,42984
1.0,11,NaN,27,184,8,66,5,171,22,17,167,23,7,23
2.0,28,2.0,80,513,16,154,4,491,47,8,337,36,13,36
3.0,96,NaN,155,930,15,354,13,6297,105,20,452,68,15,34
4.0,166,2.0,139,1092,6,346,25,9264,87,36,372,118,6,17
5.0,2017,54.0,142,654,63,213,2769,6429,2511,41454,275,5935,47,38
6.0,37,1.0,28,183,5,43,40,1323,72,212,53,176,3,2


In [89]:
# Cálculos para comprobaciones (para cada idioma, con su código numérico como valor de n):
n=4 # (gallego)
# nC2= núm. registros idioma n con acreditación C2:
nC2=df['idioma1'][(df['idioma1']==n) & (df['idioma1_acred']==6)].count()\
  +df['idioma2'][(df['idioma2']==n) & (df['idioma2_acred']==6)].count()\
  +df['idioma3'][(df['idioma3']==n) & (df['idioma3_acred']==6)].count()\
  +df['idioma4'][(df['idioma4']==n) & (df['idioma4_acred']==6)].count() 
# nC1= núm. registros idioma n con acreditación C1:
nC1=df['idioma_m1'][df['idioma_m1']==n].count()+df['idioma_m2'][df['idioma_m2']==n].count()\
  +df['idioma1'][(df['idioma1']==n) & (df['idioma1_acred']==5)].count()\
  +df['idioma2'][(df['idioma2']==n) & (df['idioma2_acred']==5)].count()\
  +df['idioma3'][(df['idioma3']==n) & (df['idioma3_acred']==5)].count()\
  +df['idioma4'][(df['idioma4']==n) & (df['idioma4_acred']==5)].count() 
# nB2= núm. registros idioma n con acreditación B2:
nB2=df['idioma1'][(df['idioma1']==n) & (df['idioma1_acred']==4)].count()\
  +df['idioma2'][(df['idioma2']==n) & (df['idioma2_acred']==4)].count()\
  +df['idioma3'][(df['idioma3']==n) & (df['idioma3_acred']==4)].count()\
  +df['idioma4'][(df['idioma4']==n) & (df['idioma4_acred']==4)].count()
# nB1= núm. registros idioma n con acreditación B1:
nB1=df['idioma1'][(df['idioma1']==n) & (df['idioma1_acred']==3)].count()\
  +df['idioma2'][(df['idioma2']==n) & (df['idioma2_acred']==3)].count()\
  +df['idioma3'][(df['idioma3']==n) & (df['idioma3_acred']==3)].count()\
  +df['idioma4'][(df['idioma4']==n) & (df['idioma4_acred']==3)].count()
# nA2= núm. registros idioma n con acreditación A2:
nA2=df['idioma1'][(df['idioma1']==n) & (df['idioma1_acred']==2)].count()\
  +df['idioma2'][(df['idioma2']==n) & (df['idioma2_acred']==2)].count()\
  +df['idioma3'][(df['idioma3']==n) & (df['idioma3_acred']==2)].count()\
  +df['idioma4'][(df['idioma4']==n) & (df['idioma4_acred']==2)].count()
# nA1= núm. registros idioma n con acreditación A1:
nA1=df['idioma1'][(df['idioma1']==n) & (df['idioma1_acred']==1)].count()\
  +df['idioma2'][(df['idioma2']==n) & (df['idioma2_acred']==1)].count()\
  +df['idioma3'][(df['idioma3']==n) & (df['idioma3_acred']==1)].count()\
  +df['idioma4'][(df['idioma4']==n) & (df['idioma4_acred']==1)].count()
nC2,nC1,nB2,nB1,nA2,nA1

(40, 2769, 25, 13, 4, 5)

In [90]:
# Diccionarios asociados a las variables 'i_acred' (i={idiomas observados})
for idioma in idiomas: 
    D_describe[str(idioma)+'_acred']='Nivel acreditado del idioma'
    D_dict[str(idioma)+'_acred']=dict()
    D_dict[str(idioma)+'_acred'][0]='No' # Sin acreditación informada
    D_dict[str(idioma)+'_acred'][1]='A1'
    D_dict[str(idioma)+'_acred'][2]='A2'
    D_dict[str(idioma)+'_acred'][3]='B1'
    D_dict[str(idioma)+'_acred'][4]='B2'
    D_dict[str(idioma)+'_acred'][5]='C1'
    D_dict[str(idioma)+'_acred'][6]='C2'

In [91]:
# Variables literales para 'i_acred' (i={idiomas}):
for idioma in idiomas: 
    df[str(idioma)+'_acred_']=df[str(idioma)+'_acred'].replace(D_dict[str(idioma)+'_acred'])
    print (idioma, df[str(idioma)+'_acred_'].unique())

Euskera ['No' 'C1' 'B2' 'B1' 'A2' 'C2' 'A1']
Rumano ['No' 'C1' 'C2' 'B2' 'A2']
Portugués ['No' 'B2' 'A2' 'B1' 'C2' 'C1' 'A1']
Francés ['No' 'B2' 'A2' 'B1' 'C1' 'A1' 'C2']
Árabe ['No' 'C1' 'B1' 'A2' 'A1' 'B2' 'C2']
Italiano ['No' 'B1' 'B2' 'A2' 'C1' 'A1' 'C2']
Gallego ['No' 'C1' 'A2' 'B2' 'C2' 'B1' 'A1']
Inglés ['B2' 'B1' 'C1' 'No' 'A2' 'C2' 'A1']
Valenciano ['No' 'B1' 'C1' 'A1' 'A2' 'B2' 'C2']
Español ['C1' 'No' 'C2' 'A1' 'B1' 'B2' 'A2']
Alemán ['No' 'B2' 'A2' 'B1' 'C1' 'A1' 'C2']
Catalán ['No' 'C1' 'B1' 'B2' 'C2' 'A1' 'A2']
Ruso ['No' 'A2' 'B1' 'C1' 'A1' 'C2' 'B2']
Chino ['No' 'B1' 'A1' 'A2' 'B2' 'C1' 'C2']


In [92]:
# Se prescinde de columnas aux.
del df['materno_1'], df['materno_2']
for idioma in idiomas: 
    del df['m1_'+str(idioma)], df['m2_'+str(idioma)]
    del df[str(idioma)+'_materno']
    for n in range(1,5):
        try: del df['nivel_aux_i'+str(n)] # solo uno por cada valor de n
        except: pass # a partir del segundo idioma del ciclo
        del df['i'+str(n)+'_'+str(idioma)]
        del df['i'+str(n)+'_'+str(idioma)+'_nivel']

In [93]:
# Se prescinde de columnas de idiomas originales que han sido transformadas
del df['idioma_m1'], df['idioma_m2']
for n in range(1,5):
    del df['idioma'+str(n)]
    del df['idioma'+str(n)+'_nivel'], df['idioma'+str(n)+'_nivel_']
    del df['idioma'+str(n)+'_acred'], df['idioma'+str(n)+'_acred_']

In [94]:
# Inspección: columnas referidas a idiomas.
col_idiomas=list(filter(lambda col:re.findall('.*idioma.*',col),df.columns))
col_idiomas

['n_idiomas', 'dbtr_idiomas', 'vbtr_idiomas', 'dbtr_idiomas_', 'vbtr_idiomas_']

In [95]:
# Columnas referidas a alguno de los idiomas registrados: 
for idioma in idiomas:
    for elemento in filter(lambda col:re.findall(str(idioma)+'.*',col),df.columns):
        col_idiomas.append(elemento)
#col_idiomas

In [96]:
# Nombres de columnas referidas a idiomas en minúsculas: 
for col in col_idiomas:
    df.rename(columns={col:col.lower()},inplace=True)
Idiomas=idiomas
idiomas=list()
for idioma in Idiomas: 
    idiomas.append(idioma.lower())
idiomas

['euskera',
 'rumano',
 'portugués',
 'francés',
 'árabe',
 'italiano',
 'gallego',
 'inglés',
 'valenciano',
 'español',
 'alemán',
 'catalán',
 'ruso',
 'chino']

In [97]:
# Inspección: columnas referidas a idiomas.
col_idiomas=list(filter(lambda col:re.findall('.*idioma.*',col),df.columns))
for idioma in idiomas:
    for elemento in filter(lambda col:re.findall(str(idioma)+'.*',col),df.columns):
        col_idiomas.append(elemento)
#col_idiomas

## Ajuste variables asociadas a familias de estudios de grado medio
Se agrupan todas las familias de estudios de grado medio en **variables ordinales**, de modo que cada columna recoja el número de estudios de grado medio de esa familia en los que está titulado el encuestado. 

In [98]:
# Inspección: 
df[['fam_gmedio1','fam_gmedio1_']].groupby('fam_gmedio1').agg(['value_counts'])

,,fam_gmedio1_
,,value_counts
fam_gmedio1,fam_gmedio1_,
0.0,No tiene,41791
1.0,No conocido,38
2.0,Marítimo-pesquera,2
3.0,Industrias alimentarias,13
4.0,Química,13
5.0,Imagen personal,29
6.0,Sanidad,230
7.0,Seguridad y medio ambiente,2


In [99]:
# Inspección:
df['fam_gmedio2_'].unique()

array(['No tiene', 'Sanidad ', 'Imagen personal ',
       'Administración y gestión ',
       'Servicios socioculturales y a la comunidad ',
       'Comercio y marketing',
       'Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado medio)',
       'Transporte y mantenimiento de vehículos ',
       'Enseñanzas artísticas profesionales', 'Imagen y sonido',
       'Hostelería y turismo ', 'Industrias alimentarias ',
       'Electricidad y electrónica', 'Artes gráficas ',
       'Instalación y mantenimiento', 'Edificación y obra civil ',
       'Química'], dtype=object)

In [100]:
# Inspección:
df['fam_gmedio3_'].unique()

array(['No tiene', 'Sanidad ',
       'Servicios socioculturales y a la comunidad ',
       'Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado medio)'],
      dtype=object)

In [101]:
# Comprobación: todas las familias en la segunda y la tercera columnas originales están presentes también en ...
# ... la primera (permite tomar la primera como base para las agregaciones de familias)
df['fam_gmedio2'].isin(df['fam_gmedio1']).unique(), df['fam_gmedio3'].isin(df['fam_gmedio1']).unique()

(array([ True]), array([ True]))

In [102]:
# Variables binarias para cada familia de estudios de grado medio en cada posición (de 1 a 3):
df=pd.get_dummies(data=df,columns=['fam_gmedio1_','fam_gmedio2_','fam_gmedio3_'],prefix=['fgm1','fgm2','fgm3'])

In [103]:
# Renombra las columnas resultantes de get_dummies prescindiendo del los textos entre paréntesis: 
D_fix=dict()
for col in list(filter(lambda col: col.startswith('fgm'),df.columns)): 
    D_fix[col]=col.split('(')[0].strip()
df.rename(columns=D_fix,inplace=True)

In [104]:
# Se prescinde de las columnas asociadas a valores 0 y 1 ('No tiene' o 'No conocido')
del df['fgm1_No tiene'], df['fgm2_No tiene'], df['fgm3_No tiene'], df['fgm1_No conocido']
try: df['fgm1_No conocido']
except: pass
try: df['fgm1_No conocido']
except: pass

In [105]:
# Lista de columnas binarias asociadas a familias de estudios de grado medio en la columna 'fam_gmedio1'...
# ... (contiene todas las familias registradas):
lista_fam_gmed_1=list(filter(lambda col: col.startswith('fgm1'),df.columns))   
lista_fam_gmed_1[:5]

['fgm1_Actividades físicas y deportivas',
 'fgm1_Administración y gestión',
 'fgm1_Artes gráficas',
 'fgm1_Comercio y marketing',
 'fgm1_Edificación y obra civil']

In [106]:
# Lista de familias de estudios de grado medio: 
lista_familias=list(map(lambda familia: familia.split('_')[1],lista_fam_gmed_1))
lista_familias[:5]

['Actividades físicas y deportivas',
 'Administración y gestión',
 'Artes gráficas',
 'Comercio y marketing',
 'Edificación y obra civil']

In [107]:
# Se recogen los valores asociados a la misma familia en todas las columnas (hasta 3) asociadas a la misma: 
for familia in lista_familias:
    df['g_medio: '+str(familia)]=df['fgm1_'+str(familia)]
    if 'fgm2_'+str(familia) in df.columns: 
        df['g_medio: '+str(familia)]=df['g_medio: '+str(familia)]+df['fgm2_'+str(familia)]
    if 'fgm3_'+str(familia) in df.columns: 
        df['g_medio: '+str(familia)]=df['g_medio: '+str(familia)]+df['fgm3_'+str(familia)]

In [108]:
col_familias_gmed=list()
for familia in lista_familias: 
    for col in list(filter(lambda col: re.findall( 'g_medio: '+str(familia),col),df.columns)):
        col_familias_gmed.append(col)
#col_familias_gmed

In [109]:
# Inspección: 
df[col_familias_gmed].agg(['value_counts'])

,g_medio: Actividades físicas y deportivas,g_medio: Administración y gestión,g_medio: Artes gráficas,g_medio: Comercio y marketing,g_medio: Edificación y obra civil,g_medio: Electricidad y electrónica,g_medio: Enseñanzas artísticas profesionales,g_medio: Fabricación mecánica,g_medio: Hostelería y turismo,g_medio: Imagen personal,g_medio: Imagen y sonido,g_medio: Industrias alimentarias,g_medio: Industrias extractivas,g_medio: Informática y comunicaciones,g_medio: Instalación y mantenimiento,"g_medio: Madera, mueble y corcho",g_medio: Marítimo-pesquera,g_medio: Química,g_medio: Sanidad,g_medio: Seguridad y medio ambiente,g_medio: Servicios socioculturales y a la comunidad,"g_medio: Textil, confección y piel",g_medio: Transporte y mantenimiento de vehículos
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,42940,42927.0,43124.0,43107.0,43124.0,43047.0,42804.0,43112.0,43099.0,43099.0,43113.0,43121.0,43133.0,43121.0,43123.0,43133.0,43132.0,43120.0,42893,43132.0,43088.0,43127.0,43114.0
1,178,205.0,8.0,27.0,9.0,85.0,320.0,22.0,32.0,32.0,21.0,12.0,1.0,13.0,11.0,1.0,2.0,14.0,227,2.0,46.0,7.0,20.0
2,15,2.0,2.0,NaN,1.0,2.0,10.0,NaN,3.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN


In [110]:
# Comprobación: 
check=cuenta_repeticiones(df,['fam_gmedio1','fam_gmedio2','fam_gmedio3'])
check.rename(columns=D_dict['fam_gmedio1'])

,No tiene,Agraria,Marítimo-pesquera,Industrias alimentarias,Química,Imagen personal,Sanidad,Seguridad y medio ambiente,Fabricación mecánica,Instalación y mantenimiento,Electricidad y electrónica,Transporte y mantenimiento de vehículos,Industrias extractivas,Edificación y obra civil,"Madera, mueble y corcho","Textil, confección y piel",Artes gráficas,Imagen y sonido,Informática y comunicaciones,Administración y gestión,Comercio y marketing,Servicios socioculturales y a la comunidad,Hostelería y turismo,Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado medio),Enseñanzas artísticas profesionales
Repetitions,,,,,,,,,,,,,,,,,,,,,,,,,
0,7,43096,43132,43121,43120,43099,42893,43132,43112,43123,43047,43114,43133,43124,43133,43127,43124,43113,43121,42927,43107,43088,43099,42940,42804
1,87,38,2,12,14,32,227,2,22,11,85,20,1,9,1,7,8,21,13,205,27,46,32,178,320
2,1249,0,0,1,0,3,12,0,0,0,2,0,0,1,0,0,2,0,0,2,0,0,3,15,10
3,41791,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [111]:
# Actualiza listas de columnas auxiliares para eliminarlas: 
lista_fam_gmed_1=list(filter(lambda col: col.startswith('fgm1'),df.columns))  
lista_fam_gmed_2=list(filter(lambda col: col.startswith('fgm2'),df.columns)) 
lista_fam_gmed_3=list(filter(lambda col: col.startswith('fgm3'),df.columns)) 

In [112]:
# Se prescinde de columnas auxiliares: 
for col in lista_fam_gmed_1+lista_fam_gmed_2+lista_fam_gmed_3:
    del df[col]

In [113]:
# Actualización de diccionarios: 
for familia in lista_familias: 
    D_describe['g_medio: '+str(familia)]='Núm. de títulos de la familia de estudios de grado medio '+str(familia)
    #print (D_describe['g_medio: '+str(familia)])

## Ajuste variables asociadas a familias de estudios de grado superior
Se agrupan todas las familias de estudios de grado superior en **variables ordinales**, de modo que cada columna recoja el número de estudios de grado superior de esa familia en los que está titulado el encuestado.  
   
El proceso de transformación es análogo al seguido para las familas de estudios de grado medio. 

In [114]:
# Inspección: 
df[['fam_gsup1','fam_gsup1_']].groupby('fam_gsup1').agg(['value_counts'])

,,fam_gsup1_
,,value_counts
fam_gsup1,fam_gsup1_,
0.0,No tiene,37877
1.0,No conocido,199
2.0,Marítimo-pesquera,24
3.0,Industrias alimentarias,36
4.0,Química,128
5.0,Imagen personal,62
6.0,Sanidad,785
7.0,Seguridad y medio ambiente,65


In [115]:
# Inspección:
df['fam_gsup2_'].unique()

array(['No tiene', 'Sanidad ', 'Comercio y marketing', 'Imagen personal ',
       'Administración y gestión ',
       'Servicios socioculturales y a la comunidad ',
       'Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)',
       'Artes gráficas ', 'Edificación y obra civil ',
       'Hostelería y turismo ', 'Informática y comunicaciones',
       'Electricidad y electrónica',
       'Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado superior)',
       'Química', 'Imagen y sonido', 'Artes y artesanías',
       'Textil, confección y piel', 'No conocido',
       'Seguridad y medio ambiente', 'Fabricación mecánica ',
       'Transporte y mantenimiento de vehículos ',
       'Instalación y mantenimiento', 'Industrias alimentarias ',
       'Marítimo-pesquera', 'Madera, mueble y corcho '], dtype=object)

In [116]:
# Inspección:
df['fam_gsup3_'].unique()

array(['No tiene', 'Servicios socioculturales y a la comunidad ',
       'Imagen y sonido',
       'Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)',
       'Edificación y obra civil ', 'No conocido', 'Sanidad ',
       'Madera, mueble y corcho ', 'Electricidad y electrónica',
       'Informática y comunicaciones', 'Instalación y mantenimiento',
       'Química', 'Industrias alimentarias ',
       'Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado superior)',
       'Comercio y marketing', 'Artes gráficas '], dtype=object)

In [117]:
# Comprobación: todas las familias en la segunda y la tercera columnas originales están presentes también en ...
# ... la primera (permite tomar la primera como base para las agregaciones de familias)
df['fam_gsup2'].isin(df['fam_gsup1']).unique(), df['fam_gsup3'].isin(df['fam_gsup1']).unique()

(array([ True]), array([ True]))

In [118]:
# Variables binarias para cada familia de estudios de grado superior en cada posición (de 1 a 3):
df=pd.get_dummies(data=df,columns=['fam_gsup1_','fam_gsup2_','fam_gsup3_'],prefix=['fgs1','fgs2','fgs3'])

In [119]:
# Renombra las columnas resultantes de get_dummies prescindiendo de los textos entre paréntesis: 
D_fix=dict()
for col in list(filter(lambda col: col.startswith('fgs'),df.columns)): 
    D_fix[col]=col.split('(')[0].strip()
df.rename(columns=D_fix,inplace=True)

In [120]:
# Se prescinde de las columnas asociadas a valores 0 y 1 ('No tiene' o 'No conocido')
del df['fgs1_No tiene'], df['fgs2_No tiene'], df['fgs3_No tiene'], df['fgs1_No conocido']
try: df['fgs1_No conocido']
except: pass
try: df['fgs1_No conocido']
except: pass

In [121]:
# Lista de columnas binarias asociadas a familias de estudios de grado superior en la columna 'fam_gsup1'...
# ... (contiene todas las familias registradas):
lista_fam_gsup_1=list(filter(lambda col: col.startswith('fgs1'),df.columns))   
lista_fam_gsup_1[:5]

['fgs1_Actividades físicas y deportivas',
 'fgs1_Administración y gestión',
 'fgs1_Artes gráficas',
 'fgs1_Artes y artesanías',
 'fgs1_Comercio y marketing']

In [122]:
# Lista de familias de estudios de grado superior: 
lista_familias=list(map(lambda familia: familia.split('_')[1],lista_fam_gsup_1))
lista_familias[:5]

['Actividades físicas y deportivas',
 'Administración y gestión',
 'Artes gráficas',
 'Artes y artesanías',
 'Comercio y marketing']

In [123]:
# Se recogen los valores asociados a la misma familia en todas las columnas (hasta 3) asociadas a la misma: 
for familia in lista_familias:
    df['g_superior: '+str(familia)]=df['fgs1_'+str(familia)]
    if 'fgs2_'+str(familia) in df.columns: 
        df['g_superior: '+str(familia)]=df['g_superior: '+str(familia)]+df['fgs2_'+str(familia)]
    if 'fgs3_'+str(familia) in df.columns: 
        df['g_superior: '+str(familia)]=df['g_superior: '+str(familia)]+df['fgs3_'+str(familia)]

In [124]:
col_familias_gsup=list(filter(lambda col: re.findall('g_superior: ',col),df.columns))
col_familias_gsup[:5]

['g_superior: Actividades físicas y deportivas',
 'g_superior: Administración y gestión',
 'g_superior: Artes gráficas',
 'g_superior: Artes y artesanías',
 'g_superior: Comercio y marketing']

In [125]:
# Inspección: 
df[col_familias_gsup].agg(['value_counts'])

,g_superior: Actividades físicas y deportivas,g_superior: Administración y gestión,g_superior: Artes gráficas,g_superior: Artes y artesanías,g_superior: Comercio y marketing,g_superior: Edificación y obra civil,g_superior: Electricidad y electrónica,g_superior: Energía y agua,g_superior: Estudios artísticos superiores,g_superior: Fabricación mecánica,g_superior: Hostelería y turismo,g_superior: Imagen personal,g_superior: Imagen y sonido,g_superior: Industrias alimentarias,g_superior: Informática y comunicaciones,g_superior: Instalación y mantenimiento,"g_superior: Madera, mueble y corcho",g_superior: Marítimo-pesquera,g_superior: Química,g_superior: Sanidad,g_superior: Seguridad y medio ambiente,g_superior: Servicios socioculturales y a la comunidad,"g_superior: Textil, confección y piel",g_superior: Transporte y mantenimiento de vehículos,g_superior: Vidrio y cerámica
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,42781,42556.0,43038,43114.0,42925.0,42943,42832,43129.0,42699,43048.0,42930.0,43069.0,42883.0,43095.0,42736,43087.0,43129.0,43105.0,43001.0,42316,43064.0,42270.0,43121.0,43080.0,43130.0
1,342,572.0,87,20.0,207.0,181,291,5.0,401,82.0,201.0,64.0,247.0,39.0,381,47.0,5.0,29.0,131.0,780,70.0,846.0,13.0,54.0,4.0
2,10,6.0,8,NaN,2.0,8,9,NaN,27,4.0,3.0,1.0,4.0,NaN,16,NaN,NaN,NaN,2.0,37,NaN,18.0,NaN,NaN,NaN
3,1,NaN,1,NaN,NaN,2,2,NaN,7,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN


In [126]:
# Comprobación: 
check=cuenta_repeticiones(df,['fam_gsup1','fam_gsup2','fam_gsup3'])
check.rename(columns=D_dict['fam_gsup1'])

,No tiene,Agraria,Marítimo-pesquera,Industrias alimentarias,Química,Imagen personal,Sanidad,Seguridad y medio ambiente,Fabricación mecánica,Instalación y mantenimiento,Electricidad y electrónica,Energía y agua,Transporte y mantenimiento de vehículos,Edificación y obra civil,Vidrio y cerámica,"Madera, mueble y corcho","Textil, confección y piel",Artes gráficas,Imagen y sonido,Informática y comunicaciones,Administración y gestión,Comercio y marketing,Servicios socioculturales y a la comunidad,Hostelería y turismo,Actividades físicas y deportivas (incluidas enseñanzas deportivas de grado superior),Artes y artesanías,"Estudios artísticos superiores (artes plásticas, diseño, música y danza, arte dramático o conservación y restauración de bienes culturales)"
Repetitions,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,37,42935,43105,43095,43001,43069,42316,43064,43048,43087,42832,43129,43080,42943,43130,43129,43121,43038,42883,42736,42556,42925,42270,42930,42781,43114,42699
1,330,192,29,39,131,64,780,70,82,47,291,5,54,181,4,5,13,87,247,381,572,207,846,201,342,20,401
2,4890,6,0,0,2,1,37,0,4,0,9,0,0,8,0,0,0,8,4,16,6,2,18,3,10,0,27
3,37877,1,0,0,0,0,1,0,0,0,2,0,0,2,0,0,0,1,0,1,0,0,0,0,1,0,7


In [127]:
# Actualiza listas de columnas auxiliares para eliminarlas: 
lista_fam_gsup_1=list(filter(lambda col: col.startswith('fgs1'),df.columns))  
lista_fam_gsup_2=list(filter(lambda col: col.startswith('fgs2'),df.columns)) 
lista_fam_gsup_3=list(filter(lambda col: col.startswith('fgs3'),df.columns)) 

In [128]:
# Se prescinde de columnas auxiliares: 
for col in lista_fam_gsup_1+lista_fam_gsup_2+lista_fam_gsup_3:
    del df[col]

In [129]:
# Se prescinde de columnas originales referidas a grados medios y superiores: 
for col in filter(lambda col: re.findall('fam',col),df.columns):
    del df[col]

In [130]:
# Actualización de diccionarios: 
for familia in lista_familias: 
    D_describe['g_superior: '+str(familia)]='Núm. de títulos de la familia de estudios de grado superior '\
    +str(familia)
    #print (D_describe['g_superior: '+str(familia)])

## Ajuste variables asociadas a ámbitos de estudio de grado universitario
Se agrupan todos los ámbitos de estudio de los títulos universitarios en **variables ordinales**, de modo que las columnas resultantes recogen el número de títulos en cada ámbito en los que está titulado el encuestado.  

In [131]:
# Inspección: 
df[['ambito_g1','ambito_g1_']].groupby('ambito_g1').agg(['value_counts'])

,,ambito_g1_
,,value_counts
ambito_g1,ambito_g1_,
0.0,No tiene,81
1.0,No conocido,331
11.0,Educación (Otros estudios),1678
21.0,Artes (Otros estudios),1262
22.0,Humanidades,1466
23.0,Lenguas,1836
31.0,Ciencias sociales y del comportamiento (Otros estudios),917
32.0,Periodismo y documentación,1026


In [132]:
# Inspección:
df['ambito_g4_'].unique()

array(['No tiene', 'Formación de docentes de enseñanza primaria ',
       'Educación (Otros estudios) ',
       'Formación de docentes de enseñanza infantil ', 'No conocido',
       'Artes (Otros estudios) ', 'Actividades físicas y deportivas ',
       'Lenguas ', 'Humanidades ', 'Periodismo y documentación ',
       'Ciencias sociales y del comportamiento (Otros estudios) ',
       'Dirección y administración ', 'Enfermería ', 'Derecho ',
       'Industria manufacturera y producción ',
       'Negocios y administración (Otros estudios) ', 'Economía ',
       'Ingeniería y profesiones afines ', 'Informática',
       'Arquitectura y construcción ', 'Silvicultura ',
       'Matemáticas y estadística ', 'Servicios (Otros estudios) ',
       'Salud (Otros estudios) ', nan], dtype=object)

In [133]:
# Comprobación: todos los ámbitos de estudio relevantes de las titulaciones de grado están presentes en la ...
# ... columna referida al primer grado universitario (permite tomarla como base para construir el algoritmo de ...
# ... agregación de ámbitos de grado).
df['ambito_g2'].isin(df['ambito_g1']).unique(), df['ambito_g3'].isin(df['ambito_g1']).unique(),\
 df['ambito_g4'].isin(df['ambito_g1']).unique()

(array([ True]), array([ True]), array([ True, False]))

In [134]:
# La categoría presente en 'ambito_g4' que no está recogida en 'ambito_g1' son no informados. 
df['ambito_g4'][~df['ambito_g4'].isin(df['ambito_g1'])].unique()

array([nan])

In [135]:
# Inspección: El criterio para agrupar las titulaciones de máster en ámbitos de estudio es distinto del ...
# ... aplicado a los títulos de grado. 
df['ambito_m1_'].unique()

array([nan, 'SALUD Y SERVICIOS SOCIALES', 'EDUCACIÓN', 'INFORMÁTICA',
       'CIENCIAS', 'INGENIERÍA, INDUSTRIA Y CONSTRUCCIÓN',
       'NEGOCIOS, ADMINISTRACIÓN Y DERECHO', 'SERVICIOS',
       'CIENCIAS SOCIALES, PERIODISMO Y DOCUMENTACIÓN',
       'ARTES Y HUMANIDADES',
       'AGRICULTURA, GANADERÍA, SILVICULTURA, PESCA Y VETERINARIA'],
      dtype=object)

In [136]:
# Variables binarias para cada ámbito de estudios de grado en cada posición (de 1 a 4):
df=pd.get_dummies(data=df,columns=['ambito_g1_','ambito_g2_','ambito_g3_','ambito_g4_'],\
                  prefix=['ag1','ag2','ag3','ag4'])

In [137]:
# Renombra las columnas resultantes de get_dummies prescindiendo de los textos entre paréntesis: 
D_fix=dict()
for col in list(filter(lambda col: col.startswith('ag'),df.columns)): 
    D_fix[col]=col.split('(')[0].strip()
df.rename(columns=D_fix,inplace=True)

In [138]:
# Se prescinde de las columnas asociadas a valores 0 y 1 ('No tiene' o 'No conocido')
del df['ag1_No tiene'], df['ag2_No tiene'], df['ag3_No tiene'], df['ag4_No tiene'], df['ag1_No conocido']
try: df['ag2_No conocido']
except: pass
try: df['ag3_No conocido']
except: pass
try: df['ag4_No conocido']
except: pass

In [139]:
# Lista de columnas binarias asociadas a ámbitos de conocimiento de los títulos de grado en la columna ...
# ... 'ambito_g1' (contiene todos los ámbitos observados):
col_ambitos_g=list(filter(lambda col: col.startswith('ag1'),df.columns))   
col_ambitos_g[:5]

['ag1_Actividades físicas y deportivas',
 'ag1_Agricultura y ganadería',
 'ag1_Arquitectura y construcción',
 'ag1_Artes',
 'ag1_Ciencias de la vida']

In [140]:
# Lista de ámbitos de conocimiento de grado: 
lista_ambitos_g=list(map(lambda familia: familia.split('_')[1],col_ambitos_g))
lista_ambitos_g[:5]

['Actividades físicas y deportivas',
 'Agricultura y ganadería',
 'Arquitectura y construcción',
 'Artes',
 'Ciencias de la vida']

In [141]:
# Se recogen los valores asociados al mismo ámbito de estudios de grado en todas las columnas (hasta 4)
for ambito in lista_ambitos_g:
    df['n_g_ambito '+str(ambito)]=df['ag1_'+str(ambito)]
    if 'ag2_'+str(ambito) in df.columns: 
        df['n_g_ambito '+str(ambito)]=df['n_g_ambito '+str(ambito)]+df['ag2_'+str(ambito)]
    if 'ag3_'+str(ambito) in df.columns: 
        df['n_g_ambito '+str(ambito)]=df['n_g_ambito '+str(ambito)]+df['ag3_'+str(ambito)]
    if 'ag4_'+str(ambito) in df.columns: 
        df['n_g_ambito '+str(ambito)]=df['n_g_ambito '+str(ambito)]+df['ag4_'+str(ambito)]

In [142]:
col_n_g_ambito=list(filter(lambda col: re.findall('n_g_ambito ',col),df.columns))
col_n_g_ambito[:5]

['n_g_ambito Actividades físicas y deportivas',
 'n_g_ambito Agricultura y ganadería',
 'n_g_ambito Arquitectura y construcción',
 'n_g_ambito Artes',
 'n_g_ambito Ciencias de la vida']

In [143]:
# Inspección: 
df[col_n_g_ambito].agg(['value_counts'])

,n_g_ambito Actividades físicas y deportivas,n_g_ambito Agricultura y ganadería,n_g_ambito Arquitectura y construcción,n_g_ambito Artes,n_g_ambito Ciencias de la vida,"n_g_ambito Ciencias químicas, físicas y geológicas",n_g_ambito Ciencias sociales y del comportamiento,n_g_ambito Derecho,n_g_ambito Dirección y administración,n_g_ambito Economía,n_g_ambito Educación,n_g_ambito Enfermería,n_g_ambito Formación de docentes de enseñanza infantil,n_g_ambito Formación de docentes de enseñanza primaria,n_g_ambito Humanidades,n_g_ambito Industria manufacturera y producción,n_g_ambito Informática,n_g_ambito Ingeniería y profesiones afines,n_g_ambito Lenguas,n_g_ambito Matemáticas y estadística,n_g_ambito Medicina,n_g_ambito Medio ambiente,n_g_ambito Negocios y administración,n_g_ambito Periodismo y documentación,n_g_ambito Psicología,n_g_ambito Salud,n_g_ambito Servicios,n_g_ambito Servicios sociales,n_g_ambito Silvicultura,n_g_ambito Técnicas audiovisuales y medios de comunicación,n_g_ambito Veterinaria,"n_g_ambito Viajes, turismo y ocio"
,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts,value_counts
0,42146.0,42296.0,40872,41783.0,41363.0,41547.0,41971.0,40412.0,39042.0,42026.0,40555.0,41404.0,41090.0,40314,41465.0,42376.0,41469.0,39119,41153,42445.0,42173.0,42369.0,42002.0,41968.0,41663.0,41107,42845.0,42085.0,42733.0,42422.0,42579.0,42281.0
1,982.0,666.0,1689,1308.0,1686.0,1571.0,1106.0,2708.0,3591.0,1105.0,2112.0,1613.0,1961.0,2631,1624.0,669.0,1357.0,3208,1837,610.0,959.0,765.0,1109.0,1101.0,1465.0,1863,265.0,938.0,325.0,710.0,549.0,801.0
2,6.0,166.0,548,41.0,85.0,15.0,56.0,14.0,486.0,3.0,443.0,117.0,82.0,178,43.0,84.0,300.0,769,138,78.0,2.0,NaN,23.0,64.0,6.0,152,24.0,110.0,76.0,2.0,6.0,50.0
3,NaN,6.0,24,2.0,NaN,1.0,1.0,NaN,15.0,NaN,24.0,NaN,1.0,9,2.0,5.0,8.0,35,5,1.0,NaN,NaN,NaN,1.0,NaN,10,NaN,1.0,NaN,NaN,NaN,2.0
4,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,3,1,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
# Comprobación: 
check=cuenta_repeticiones(df,['ambito_g1','ambito_g2','ambito_g3','ambito_g4'])
check.rename(columns=D_dict['ambito_g1'])

,No tiene,No conocido,Educación (Otros estudios),Medicina,Enfermería,Artes (Otros estudios),Humanidades,Lenguas,Dirección y administración,Ciencias sociales y del comportamiento (Otros estudios),Periodismo y documentación,Negocios y administración (Otros estudios),Derecho,Ciencias de la vida,Medio ambiente,"Ciencias químicas, físicas y geológicas",Matemáticas y estadística,Economía,Psicología,Informática,Ingeniería y profesiones afines,Industria manufacturera y producción,Arquitectura y construcción,Agricultura y ganadería,Silvicultura,Técnicas audiovisuales y medios de comunicación,Veterinaria,Salud (Otros estudios),Servicios sociales,Servicios (Otros estudios),Formación de docentes de enseñanza infantil,Formación de docentes de enseñanza primaria,Actividades físicas y deportivas,"Viajes, turismo y ocio",nan
Repetitions,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,377,42704,40555,42173,41404,41783,41465,41153,39042,41971,41968,42002,40412,41363,42369,41547,42445,42026,41663,41469,39119,42376,40872,42296,42733,42422,42579,41107,42085,42845,41090,40314,42146,42281,31651
1,2949,399,2112,959,1613,1308,1624,1837,3591,1106,1101,1109,2708,1686,765,1571,610,1105,1465,1357,3208,669,1689,666,325,710,549,1863,938,265,1961,2631,982,801,11483
2,15498,27,443,2,117,41,43,138,486,56,64,23,14,85,0,15,78,3,6,300,769,84,548,166,76,2,6,152,110,24,82,178,6,50,0
3,24310,4,24,0,0,2,2,5,15,1,1,0,0,0,0,1,1,0,0,8,35,5,24,6,0,0,0,10,1,0,1,9,0,2,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,2,0,0,0,2,0,0,0


In [145]:
# Actualiza listas de columnas auxiliares para eliminarlas: 
lista_ambito_1=list(filter(lambda col: col.startswith('ag1_'),df.columns))  
lista_ambito_2=list(filter(lambda col: col.startswith('ag2_'),df.columns)) 
lista_ambito_3=list(filter(lambda col: col.startswith('ag3_'),df.columns)) 
lista_ambito_4=list(filter(lambda col: col.startswith('ag4_'),df.columns)) 

In [146]:
# Se prescinde de columnas auxiliares: 
for col in lista_ambito_1+lista_ambito_2+lista_ambito_3+lista_ambito_4:
    del df[col]

In [147]:
# Se prescinde de columnas originales referidas a grados medios y superiores: 
for col in filter(lambda col: re.findall('ambito_g',col),df.columns):
    del df[col]

In [148]:
# Actualización de diccionarios: 
for ambito in lista_ambitos_g: 
    D_describe['n_g_ambito '+str(ambito)]='Núm. de títulos de grado del ámbito de estudios '\
    +str(ambito)
    #print (D_describe['n_g_ambito '+str(ambito)])

## Diccionario abreviado de ocupaciones

In [149]:
# Diccionario abreviado de ocupaciones:
D_dict['ocupacion']=dict()
D_dict['ocupacion'][0]='Ocupaciones militares'
D_dict['ocupacion'][11]='Política y dirección Admón Pública'
D_dict['ocupacion'][12]='Directores Admón y comerciales'
D_dict['ocupacion'][13]='Directores producción y operaciones'
D_dict['ocupacion'][14]='Gerentes hostelería y comercio'
D_dict['ocupacion'][15]='Gerentes em servicios excepto hostelería y comercio'
D_dict['ocupacion'][21]='Prof sanitarios'
D_dict['ocupacion'][22]='Prof enseñanza'
D_dict['ocupacion'][23]='Prof eduación especial y no reglada'
D_dict['ocupacion'][24]='Prof ciencias, matemáticas e ingeniería'
D_dict['ocupacion'][25]='Prof derecho'
D_dict['ocupacion'][26]='Expertos en gestión, finanzas, comercio, etc'
D_dict['ocupacion'][27]='Profesionales IT'
D_dict['ocupacion'][28]='Prof ciencias sociales'
D_dict['ocupacion'][29]='Prof cultura y espectáculo'
D_dict['ocupacion'][31]='Técnicos ciencias e ingenierías'
D_dict['ocupacion'][32]='Supervisores ingeniería, industria y construcción'
D_dict['ocupacion'][33]='Técnicos sanitarios y laboratorio'
D_dict['ocupacion'][34]='Prof apoyo finanzas y matemáticas'
D_dict['ocupacion'][35]='Representantes e intermediarios'
D_dict['ocupacion'][36]='Prof apoyo gestión administrativa'
D_dict['ocupacion'][37]='Prof apoyo servicios jurídicos, socioculturales, deportivos'
D_dict['ocupacion'][38]='Técnicos TI'
D_dict['ocupacion'][41]='Financieros, contables, etc'
D_dict['ocupacion'][42]='Empleados bibliotecas, correos, etc'
D_dict['ocupacion'][43]='Empleados administrativos sin atención al público'
D_dict['ocupacion'][44]='Empleos atención al público'
D_dict['ocupacion'][45]='Empleados administrativos con atención al público'
D_dict['ocupacion'][50]='Camareros y cocineros propietarios'
D_dict['ocupacion'][51]='Asalariados servicios de restauración'
D_dict['ocupacion'][52]='Dependientes de comercio'
D_dict['ocupacion'][53]='Comerciantes propietarios de tiendas '
D_dict['ocupacion'][54]='Vendedores excepto comercio'
D_dict['ocupacion'][55]='Cajeros y taquilleros excepto bancos'
D_dict['ocupacion'][56]='Trabajadores servicios de cuidados de salud'
D_dict['ocupacion'][57]='Trabajadores servicios de cuidados no de salud'
D_dict['ocupacion'][58]='Trabajadores servicios personales'
D_dict['ocupacion'][59]='Trabajadores protección y seguridad'
D_dict['ocupacion'][61]='Trabajadores cualificados agricultura'
D_dict['ocupacion'][62]='Trabajadores cualificados ganadería'
D_dict['ocupacion'][63]='Trabajadores cualificados act. agropecuaria'
D_dict['ocupacion'][64]='Trabajadores cualif act forestales, pesqueras, cinegéticas'
D_dict['ocupacion'][71]='Trabajadores construcción y afines: estructura'
D_dict['ocupacion'][72]='Trabajadores construcción e instalaciones: acabado'
D_dict['ocupacion'][73]='Soldadores, chapistas, montadores de estructuras'
D_dict['ocupacion'][74]='Mecánicos y ajustadores de maquinaria'
D_dict['ocupacion'][75]='Trabajadores especializados electricidad y electrotecnología'
D_dict['ocupacion'][76]='Mecánicos de precisión, artesanos y artes gráficas'
D_dict['ocupacion'][77]='Trabajadores ind alimentación, bebidas y tabaco'
D_dict['ocupacion'][78]='Trabajadores madera, textil, confección, piel y cuero'
D_dict['ocupacion'][81]='Operadores de instalaciones y maquinaria fija'
D_dict['ocupacion'][82]='Montadores y ensambladores en fábricas'
D_dict['ocupacion'][83]='Maquinistas de equipos pesados móviles'
D_dict['ocupacion'][84]='Conductores vehículos transporte urbano o carretera'
D_dict['ocupacion'][91]='Empleados domésticos'
D_dict['ocupacion'][92]='Personal de limpieza de establecimientos y vehículos'
D_dict['ocupacion'][93]='Ayudantes de preparación de alimentos'
D_dict['ocupacion'][94]='Recogedores residuos urbanos y ocupaciones elementales servicios'
D_dict['ocupacion'][95]='Peones agrarios, forestales y pesca'
D_dict['ocupacion'][96]='Peones de construcción y minería'
D_dict['ocupacion'][97]='Peones de industrias manufactureras'
D_dict['ocupacion'][98]='Peones de transporte, descargadores y reponedores'

In [150]:
df['ocupacion_']=df['ocupacion'].replace(D_dict['ocupacion'])
df['ocupacion_'].unique().tolist()

['Prof enseñanza',
 nan,
 'Prof eduación especial y no reglada',
 'Empleados administrativos con atención al público',
 'Empleos atención al público',
 'Trabajadores servicios personales',
 'Expertos en gestión, finanzas, comercio, etc',
 'Prof ciencias sociales',
 'Peones de transporte, descargadores y reponedores',
 'Técnicos sanitarios y laboratorio',
 'Empleados bibliotecas, correos, etc',
 'Asalariados servicios de restauración',
 'Técnicos TI',
 'Trabajadores servicios de cuidados de salud',
 'Gerentes hostelería y comercio',
 'Prof apoyo servicios jurídicos, socioculturales, deportivos',
 'Dependientes de comercio',
 'Representantes e intermediarios',
 'Prof sanitarios',
 'Directores producción y operaciones',
 'Trabajadores servicios de cuidados no de salud',
 'Prof apoyo gestión administrativa',
 'Empleados administrativos sin atención al público',
 'Personal de limpieza de establecimientos y vehículos',
 'Trabajadores protección y seguridad',
 'Financieros, contables, etc',
 

## Inspección de filtros por 'sit_lab',  'motivacion' y 'utilidad'

In [151]:
D_describe['motivacion']

'Motivo principal por el que ha realizado sus estudios'

In [152]:
df[['motivacion','motivacion_']].value_counts(normalize=True)

motivacion  motivacion_                                    
1.0         Formarse pensando en su futuro laboral             0.754540
2.0         Ampliar conocimientos por satisfacción personal    0.158563
3.0         Otros motivos                                      0.086897
dtype: float64

In [153]:
D_describe['utilidad']

'Cree que el título universitario le ha servido para encontrar trabajo'

In [154]:
df[['utilidad','utilidad_']].value_counts(normalize=True)

utilidad  utilidad_
1.0       Sí           0.671946
0.0       No           0.328054
dtype: float64

In [155]:
df[['sit_lab','sit_lab_']].value_counts(normalize=True)

sit_lab  sit_lab_    
1.0      Trabajando      0.862614
2.0      En desempleo    0.074674
3.0      Inactivo        0.062712
dtype: float64

In [156]:
# Observaciones perdidas filtro situación laboral = ocupado: 
df[df['sit_lab']==1].shape[0], df.shape[0]-df[df['sit_lab']==1].shape[0]

(37208, 5926)

In [157]:
# Pérdida adicional de observaciones con filtros por 'utilidad' y 'motivacion':
prd=df[df['sit_lab']==1].shape[0]-df[(df['sit_lab']==1) & (df['motivacion']==1) & (df['utilidad']==1)].shape[0]
prd, prd/df[df['sit_lab']==1].shape[0]

(16898, 0.45414964523758333)

In [164]:
acadm_binarios=['grado','master','doctor','est_art','g_sup','g_medio']
n_titulos=['n_grados','n_master','n_gmedio','n_gsup']
titulos=['titulo_ppal']
ambitos=['ambito_g1','ambito_g2','ambito_g3','ambito_g4','ambito_m1']
n_ramas=['n_art_hum','n_ciencias','n_cc_soc_ju','n_ing_arq','n_cc_salud']
g_medio=['g_medio: Actividades físicas y deportivas','g_medio: Administración y gestión',\
         'g_medio: Artes gráficas','g_medio: Comercio y marketing','g_medio: Edificación y obra civil',\
         'g_medio: Electricidad y electrónica','g_medio: Enseñanzas artísticas profesionales',\
         'g_medio: Fabricación mecánica','g_medio: Hostelería y turismo','g_medio: Imagen personal',\
         'g_medio: Imagen y sonido','g_medio: Industrias alimentarias','g_medio: Industrias extractivas',\
         'g_medio: Informática y comunicaciones','g_medio: Instalación y mantenimiento',\
         'g_medio: Madera, mueble y corcho','g_medio: Marítimo-pesquera','g_medio: Química','g_medio: Sanidad',\
         'g_medio: Seguridad y medio ambiente','g_medio: Servicios socioculturales y a la comunidad',\
         'g_medio: Textil, confección y piel','g_medio: Transporte y mantenimiento de vehículos']
g_superior=['g_superior: Actividades físicas y deportivas','g_superior: Administración y gestión',\
          'g_superior: Artes gráficas','g_superior: Artes y artesanías','g_superior: Comercio y marketing',\
          'g_superior: Edificación y obra civil','g_superior: Electricidad y electrónica',\
          'g_superior: Energía y agua','g_superior: Estudios artísticos superiores',\
          'g_superior: Fabricación mecánica','g_superior: Hostelería y turismo','g_superior: Imagen personal',\
          'g_superior: Imagen y sonido','g_superior: Industrias alimentarias',\
          'g_superior: Informática y comunicaciones','g_superior: Instalación y mantenimiento',\
          'g_superior: Madera, mueble y corcho','g_superior: Marítimo-pesquera','g_superior: Química',\
          'g_superior: Sanidad','g_superior: Seguridad y medio ambiente',\
          'g_superior: Servicios socioculturales y a la comunidad','g_superior: Textil, confección y piel',\
          'g_superior: Transporte y mantenimiento de vehículos','g_superior: Vidrio y cerámica']
idiomas=['n_idiomas','catalán','catalán_nivel','catalán_acred', 'ruso','ruso_nivel','ruso_acred','euskera',\
         'euskera_nivel','euskera_acred','chino','chino_nivel','chino_acred','rumano','rumano_nivel',\
         'rumano_acred','valenciano','valenciano_nivel','valenciano_acred','español','español_nivel',\
         'español_acred','inglés','inglés_nivel','inglés_acred','gallego','gallego_nivel','gallego_acred',\
         'francés','francés_nivel','francés_acred','portugués','portugués_nivel','portugués_acred','italiano',\
         'italiano_nivel','italiano_acred','árabe','árabe_nivel','árabe_acred','alemán','alemán_nivel',\
         'alemán_acred']
ac_subj=['motivacion','sat_est_u','sat_titulo','vbtr_idiomas']
ac_resto=['beca','beca_general','beca_excel','beca_colab','beca_pract','beca_extr','extr_proxy','extr',\
          'tipo_beca_extr','pais_beca_extr','ti']
personales=['sexo','edad','nacionalidad','eu27','discapacidad']
sociales=['tipo_centro','tipo_docencia','reside','origen_pa','estudios_pa','grad_med_pa','univ_pa','origen_ma',\
          'estudios_ma','grad_med_ma','univ_ma','extr_sin_beca']
otros=['etr_opo','etr_contactos']
regresores=acadm_binarios+n_titulos+['titulo_ppal']+ambitos+n_ramas+g_medio+g_superior+idiomas+ac_subj+ac_resto\
            +personales+sociales+otros

In [159]:
df.drop(['año','est_encuesta'],axis=1).columns

Index(['sexo', 'edad', 'nacionalidad', 'eu27', 'titulo_g1', 'rama_g1',
       'n_art_hum', 'n_ciencias', 'n_cc_soc_ju', 'n_ing_arq',
       ...
       'n_g_ambito Negocios y administración',
       'n_g_ambito Periodismo y documentación', 'n_g_ambito Psicología',
       'n_g_ambito Salud', 'n_g_ambito Servicios',
       'n_g_ambito Servicios sociales', 'n_g_ambito Silvicultura',
       'n_g_ambito Técnicas audiovisuales y medios de comunicación',
       'n_g_ambito Veterinaria', 'n_g_ambito Viajes, turismo y ocio'],
      dtype='object', length=401)

In [160]:
for col1 in df.drop(['año','est_encuesta'],axis=1).select_dtypes('number').columns:
    for col2 in df.drop(['año','est_encuesta',str(col1)],axis=1).select_dtypes('number').columns: 
        if abs(df[[col1,col2]].corr().loc[col1,col2])>0.5:
            print (col1,'\t', col2,'\t', df[[col1,col2]].corr().loc[col1,col2])

eu27 	 origen_pa 	 -0.6346595575426032
eu27 	 origen_ma 	 -0.6575752691303229
titulo_g1 	 titulo_m1 	 -0.5548147551061323
titulo_g1 	 ambito_m1 	 -0.5488786729476026
titulo_g1 	 titulo_ppal 	 0.999977051317617
rama_g1 	 n_art_hum 	 -0.578576809895529
n_art_hum 	 rama_g1 	 -0.578576809895529
n_art_hum 	 n_g_ambito Lenguas 	 0.5420807806250884
n_ing_arq 	 n_g_ambito Ingeniería y profesiones afines 	 0.5733191605171911
reside 	 pais_tr 	 0.9642308384166679
origen_pa 	 eu27 	 -0.6346595575426032
origen_pa 	 origen_ma 	 0.619313172330997
estudios_pa 	 grad_med_pa 	 0.8681607605462968
estudios_pa 	 univ_pa 	 0.811645774140827
estudios_pa 	 estudios_ma 	 0.6155225344319439
estudios_pa 	 grad_med_ma 	 0.5273376264330927
grad_med_pa 	 estudios_pa 	 0.8681607605462968
grad_med_pa 	 univ_pa 	 0.5766792362861403
grad_med_pa 	 estudios_ma 	 0.5042018108718128
univ_pa 	 estudios_pa 	 0.811645774140827
univ_pa 	 grad_med_pa 	 0.5766792362861403
univ_pa 	 estudios_ma 	 0.5043061771217597
origen_ma 	 e

rumano_acred 	 rumano 	 0.8127199858177151
rumano_acred 	 rumano_nivel 	 0.8868247937797877
portugués_acred 	 portugués_nivel 	 0.5911839203797401
francés_acred 	 francés_nivel 	 0.6464845834829368
árabe_acred 	 árabe 	 0.715874251426786
árabe_acred 	 árabe_nivel 	 0.878834204826436
italiano_acred 	 italiano 	 0.5326292298903454
italiano_acred 	 italiano_nivel 	 0.6274894775601859
gallego_acred 	 gallego 	 0.942378684679755
gallego_acred 	 gallego_nivel 	 0.9636145981705537
inglés_acred 	 inglés_nivel 	 0.5103202324977097
valenciano_acred 	 valenciano 	 0.9032347266661699
valenciano_acred 	 valenciano_nivel 	 0.9355643078821191
alemán_acred 	 alemán 	 0.6684238528517388
alemán_acred 	 alemán_nivel 	 0.778834288181046
catalán_acred 	 catalán 	 0.9115164012443479
catalán_acred 	 catalán_nivel 	 0.9411493778697266
ruso_acred 	 ruso 	 0.6401275059357018
ruso_acred 	 ruso_nivel 	 0.7968093888859702
chino_acred 	 chino 	 0.7131673277632896
chino_acred 	 chino_nivel 	 0.854818093802264
g_supe

In [165]:
for col1 in regresores:
    for col2 in [col for col in regresores if col!=col1]: 
        if abs(df[[col1,col2]].corr().loc[col1,col2])>0.5:
            print (col1,'\t', col2,'\t', df[[col1,col2]].corr().loc[col1,col2])

KeyError: "['ambito_g1'] not in index"

In [ ]:
def fix_ix(df):
    """Elimina de los nombres de columnas de dataframe 'df' la porción que sige al punto. 
    Usada para corregir el formato de los nombres de columna que resultan de pd.get_dummies sobre variables 
    tipo float.
    """
    D_ix_int=dict()
    for col in df.columns: D_ix_int[col]=col.split('.')[0]
    df.rename(columns=D_ix_int,inplace=True)
    return

In [ ]:
X=df[regresores+['ocupacion_','cnae_','pais_tr_','sueldo','jornada']]
X=pd.get_dummies(data=X,columns=['titulo_ppal_','ocupacion_','cnae_','pais_tr_'],prefix=['tt','oc','cnae','ptr'])
fix_ix(X)

In [ ]:
lista_titulos=list(filter(lambda col: col.startswith('tt'),X.columns))
lista_ocupaciones=list(filter(lambda col: col.startswith('oc'),X.columns))
lista_cnae=list(filter(lambda col: col.startswith('cnae'),X.columns))
lista_paises=list(filter(lambda col: col.startswith('ptr'),X.columns))
lista_ocupaciones

In [ ]:
for rg in lista_titulos: 
    for target in lista_ocupaciones: 
        if abs(X[[target,rg]].corr().loc[target,rg])>0.1:
            print (X[[target,rg]].corr().loc[target,rg],'\t',target,'\n\t\t\t', rg)
            print ('____________________________________________')

In [ ]:
for rg in idiomas: 
    for target in lista_ocupaciones: 
        if abs(X[[target,rg]].corr().loc[target,rg])>0.1:
            print (X[[target,rg]].corr().loc[target,rg],'\t',target,'\n\t\t\t', rg)
            print ('____________________________________________')

In [ ]:
for rg in g_medio: 
    for target in lista_ocupaciones: 
        if abs(X[[target,rg]].corr().loc[target,rg])>0.05:
            print (X[[target,rg]].corr().loc[target,rg],'\t',target,'\n\t\t\t', rg)
            print ('____________________________________________')

In [ ]:
for rg in g_superior: 
    for target in lista_ocupaciones: 
        if abs(X[[target,rg]].corr().loc[target,rg])>0.1:
            print (X[[target,rg]].corr().loc[target,rg],'\t',target,'\n\t\t\t', rg)
            print ('____________________________________________')

In [ ]:
for rg in n_titulos: 
    for target in lista_ocupaciones: 
        if abs(X[[target,rg]].corr().loc[target,rg])>0.05:
            print (X[[target,rg]].corr().loc[target,rg],'\t',target,'\n\t\t\t', rg)
            print ('____________________________________________')

In [ ]:
for rg in [rgs for rgs in regresores if rgs!='titulo_ppal_']+lista_titulos: 
    for target in lista_ocupaciones: 
        if abs(X[[target,rg]].corr().loc[target,rg])>0.2:
            print (X[[target,rg]].corr().loc[target,rg],'\t',target,'\n\t\t\t', rg)
            print ('____________________________________________')

In [ ]:
for rg in lista_cnae: 
    for target in lista_ocupaciones: 
        if abs(X[[target,rg]].corr().loc[target,rg])>0.2:
            print (X[[target,rg]].corr().loc[target,rg],'\t',target,'\n\t\t\t', rg)
            print ('____________________________________________')

## Archivo csv

In [ ]:
df.to_csv('data/formacion_procesado.csv')